# Word Graph Analysis
## as introduced by Mota et al. 2012
 make sure that all excel tables are in the same folder as this jupyter notebook if you intend to execute cells.

In [1]:
# imports packages, read in all excel tables, define classifiers and computation of results/report
import re
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut, ShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
df_movingwindows = pd.read_excel("df_movingwindows.xlsx")
df_parameters = pd.read_excel("df_parameters.xlsx")
df_parameters_task1 = pd.read_excel("df_parameters_task1.xlsx")
df_parameters_task2 = pd.read_excel("df_parameters_task2.xlsx")
df_parameters_windows_task1 = pd.read_excel("df_parameters_windows_task1.xlsx")
df_parameters_windows_task2 = pd.read_excel("df_parameters_windows_task2.xlsx")
df_parameters_windows_both = df_parameters_windows_task1 + df_parameters_windows_task2

df_movingwindows = df_movingwindows.drop("Unnamed: 0", axis=1)
df_parameters = df_parameters.drop("Unnamed: 0", axis=1)
df_parameters_task1 = df_parameters_task1.drop("Unnamed: 0", axis=1)
df_parameters_task2 = df_parameters_task2.drop("Unnamed: 0", axis=1)
df_parameters_windows_task1 = df_parameters_windows_task1.drop("Unnamed: 0", axis=1)
df_parameters_windows_task2 = df_parameters_windows_task2.drop("Unnamed: 0", axis=1)
df_parameters_windows_both = df_parameters_windows_both.drop("Unnamed: 0", axis=1)

df = pd.read_excel("df_with_lemmata.xlsx")
df = df.set_index('ID')
filenames = ['0C021_0', '0C042_0', '0C071_0', '0C081_0', '0C092_0', '0C102_0', '0C132_0', '0C151_0', '0C161_0', '0C171_0', '0C181_0', '0C201_0', '0C211_0', '0C221_0', '0C251_0', '0C271_0', '0C281_0', '0C291_0', '0C301_0', '0C311_0', '0C321_0', '0C341_0', '0C361_0', '0C371_0', '0C381_0', '0C391_0', '0C401_0', '0C411_0', '0C421_0', '0C431_0', '0C441_0', '0C471_0', '0C481_0', '0C521_0', '0C551_0', '0C561_0', '0C601_0', '0C611_0', '0C621_0', '0C631_0', '0C641_0', '0C651_0', '0C671_0', '0C681_0', '0C691_0', '0C711_0', '0C721_0', '0C731_0', '0C741_0', '0C771_0', '0P021_1', '0P031_1', '0P041_1', '0P051_1', '0P061_1', '0P091_1', '0P112_2', '0P131_1', '0P142_2', '0P161_1', '0P211_1', '0P221_1', '0P261_1', '0P391_1', '0P411_1', '0P431_1', '0P441_1', '0P451_1', '0P461_1', '0P471_1', '0P501_1', '0P541_1', '0P581_1', '0P641_1', '0P681_1', '0P691_1', '0P761_1', '100C161_0', '100C221_0', '100C271_0', '100C301_0', '100C311_0', '100C371_0', '100C391_0', '100C401_0', '100C441_0', '100C471_0', '100P161_1', '100P211_1', '100P451_1', '100P471_1', '100P581_1', '101C161_0', '101C221_0', '101C271_0', '101C301_0', '101C311_0', '101C371_0', '101C391_0', '101C401_0', '101C441_0', '101C471_0', '101P161_1', '101P211_1', '101P451_1', '101P471_1', '101P581_1', '102C161_0', '102C221_0', '102C271_0', '102C301_0', '102C311_0', '102C371_0', '102C391_0', '102C401_0', '102C441_0', '102C471_0', '102P161_1', '102P211_1', '102P451_1', '102P581_1', '103C161_0', '103C221_0', '103C271_0', '103C301_0', '103C311_0', '103C371_0', '103C391_0', '103C401_0', '103C441_0', '103C471_0', '103P161_1', '103P211_1', '103P451_1', '103P581_1', '104C161_0', '104C221_0', '104C271_0', '104C301_0', '104C311_0', '104C371_0', '104C391_0', '104C401_0', '104C441_0', '104C471_0', '104P161_1', '104P211_1', '104P451_1', '104P581_1', '105C161_0', '105C221_0', '105C271_0', '105C301_0', '105C311_0', '105C371_0', '105C391_0', '105C401_0', '105C441_0', '105C471_0', '105P161_1', '105P211_1', '105P451_1', '105P581_1', '106C161_0', '106C221_0', '106C271_0', '106C301_0', '106C311_0', '106C371_0', '106C391_0', '106C401_0', '106C441_0', '106C471_0', '106P161_1', '106P211_1', '106P451_1', '106P581_1', '107C161_0', '107C221_0', '107C271_0', '107C301_0', '107C311_0', '107C371_0', '107C391_0', '107C401_0', '107C441_0', '107C471_0', '107P161_1', '107P211_1', '107P451_1', '107P581_1', '108C161_0', '108C221_0', '108C271_0', '108C301_0', '108C311_0', '108C371_0', '108C391_0', '108C401_0', '108C441_0', '108C471_0', '108P161_1', '108P211_1', '108P451_1', '108P581_1', '109C161_0', '109C221_0', '109C271_0', '109C301_0', '109C311_0', '109C371_0', '109C391_0', '109C401_0', '109C441_0', '109C471_0', '109P161_1', '109P211_1', '109P451_1', '109P581_1', '10C021_0', '10C042_0', '10C071_0', '10C081_0', '10C092_0', '10C102_0', '10C151_0', '10C161_0', '10C171_0', '10C181_0', '10C201_0', '10C211_0', '10C221_0', '10C251_0', '10C271_0', '10C281_0', '10C291_0', '10C301_0', '10C311_0', '10C321_0', '10C341_0', '10C371_0', '10C381_0', '10C391_0', '10C401_0', '10C421_0', '10C431_0', '10C441_0', '10C471_0', '10C481_0', '10C521_0', '10C551_0', '10C561_0', '10C601_0', '10C611_0', '10C621_0', '10C631_0', '10C641_0', '10C671_0', '10C681_0', '10C691_0', '10C711_0', '10C721_0', '10C731_0', '10C741_0', '10C771_0', '10P021_1', '10P031_1', '10P041_1', '10P051_1', '10P061_1', '10P091_1', '10P112_2', '10P142_2', '10P161_1', '10P211_1', '10P221_1', '10P261_1', '10P391_1', '10P431_1', '10P441_1', '10P451_1', '10P471_1', '10P501_1', '10P541_1', '10P581_1', '10P641_1', '10P681_1', '10P761_1', '110C161_0', '110C221_0', '110C271_0', '110C301_0', '110C311_0', '110C371_0', '110C391_0', '110C401_0', '110C441_0', '110C471_0', '110P161_1', '110P211_1', '110P451_1', '110P581_1', '111C161_0', '111C221_0', '111C271_0', '111C301_0', '111C311_0', '111C371_0', '111C391_0', '111C401_0', '111C441_0', '111C471_0', '111P161_1', '111P211_1', '111P451_1', '111P581_1', '112C161_0', '112C221_0', '112C271_0', '112C301_0', '112C311_0', '112C371_0', '112C391_0', '112C401_0', '112C441_0', '112C471_0', '112P161_1', '112P211_1', '112P451_1', '112P581_1', '113C161_0', '113C221_0', '113C271_0', '113C301_0', '113C311_0', '113C371_0', '113C391_0', '113C401_0', '113C441_0', '113C471_0', '113P161_1', '113P211_1', '113P451_1', '113P581_1', '114C161_0', '114C221_0', '114C271_0', '114C301_0', '114C311_0', '114C371_0', '114C391_0', '114C401_0', '114C441_0', '114C471_0', '114P161_1', '114P211_1', '114P451_1', '114P581_1', '115C161_0', '115C221_0', '115C271_0', '115C301_0', '115C311_0', '115C371_0', '115C391_0', '115C401_0', '115C441_0', '115C471_0', '115P161_1', '115P211_1', '115P451_1', '115P581_1', '116C161_0', '116C221_0', '116C271_0', '116C301_0', '116C311_0', '116C371_0', '116C391_0', '116C401_0', '116C441_0', '116C471_0', '116P161_1', '116P211_1', '116P451_1', '116P581_1', '117C161_0', '117C221_0', '117C271_0', '117C301_0', '117C311_0', '117C371_0', '117C391_0', '117C401_0', '117C441_0', '117C471_0', '117P161_1', '117P211_1', '117P451_1', '117P581_1', '118C161_0', '118C221_0', '118C271_0', '118C301_0', '118C311_0', '118C371_0', '118C391_0', '118C401_0', '118C441_0', '118C471_0', '118P161_1', '118P211_1', '118P451_1', '118P581_1', '119C161_0', '119C221_0', '119C271_0', '119C301_0', '119C311_0', '119C371_0', '119C391_0', '119C401_0', '119C441_0', '119C471_0', '119P161_1', '119P211_1', '119P451_1', '119P581_1', '11C021_0', '11C042_0', '11C071_0', '11C081_0', '11C092_0', '11C102_0', '11C151_0', '11C161_0', '11C171_0', '11C181_0', '11C201_0', '11C211_0', '11C221_0', '11C251_0', '11C271_0', '11C281_0', '11C291_0', '11C301_0', '11C311_0', '11C321_0', '11C341_0', '11C371_0', '11C381_0', '11C391_0', '11C401_0', '11C421_0', '11C431_0', '11C441_0', '11C471_0', '11C481_0', '11C521_0', '11C551_0', '11C561_0', '11C601_0', '11C611_0', '11C621_0', '11C631_0', '11C641_0', '11C671_0', '11C681_0', '11C691_0', '11C711_0', '11C721_0', '11C731_0', '11C741_0', '11C771_0', '11P021_1', '11P031_1', '11P041_1', '11P051_1', '11P061_1', '11P091_1', '11P112_2', '11P142_2', '11P161_1', '11P211_1', '11P221_1', '11P261_1', '11P391_1', '11P431_1', '11P441_1', '11P451_1', '11P471_1', '11P501_1', '11P541_1', '11P581_1', '11P641_1', '11P681_1', '11P761_1', '120C161_0', '120C221_0', '120C271_0', '120C301_0', '120C311_0', '120C371_0', '120C391_0', '120C401_0', '120C441_0', '120C471_0', '120P161_1', '120P211_1', '120P451_1', '120P581_1', '121C161_0', '121C221_0', '121C271_0', '121C301_0', '121C311_0', '121C371_0', '121C391_0', '121C401_0', '121C441_0', '121C471_0', '121P161_1', '121P211_1', '121P451_1', '121P581_1', '122C161_0', '122C221_0', '122C271_0', '122C301_0', '122C311_0', '122C371_0', '122C391_0', '122C401_0', '122C441_0', '122C471_0', '122P161_1', '122P211_1', '122P451_1', '122P581_1', '123C161_0', '123C221_0', '123C271_0', '123C301_0', '123C311_0', '123C371_0', '123C391_0', '123C401_0', '123C441_0', '123C471_0', '123P161_1', '123P211_1', '123P451_1', '123P581_1', '124C161_0', '124C221_0', '124C271_0', '124C311_0', '124C371_0', '124C391_0', '124C401_0', '124C441_0', '124C471_0', '124P451_1', '124P581_1', '125C161_0', '125C221_0', '125C271_0', '125C311_0', '125C371_0', '125C391_0', '125C401_0', '125C441_0', '125C471_0', '125P451_1', '125P581_1', '126C161_0', '126C221_0', '126C271_0', '126C311_0', '126C371_0', '126C391_0', '126C401_0', '126C441_0', '126C471_0', '126P451_1', '126P581_1', '127C161_0', '127C221_0', '127C271_0', '127C311_0', '127C371_0', '127C391_0', '127C401_0', '127C441_0', '127C471_0', '127P451_1', '127P581_1', '128C161_0', '128C221_0', '128C271_0', '128C311_0', '128C371_0', '128C391_0', '128C401_0', '128C441_0', '128C471_0', '128P451_1', '129C161_0', '129C221_0', '129C271_0', '129C311_0', '129C371_0', '129C391_0', '129C401_0', '129C441_0', '129C471_0', '129P451_1', '12C021_0', '12C042_0', '12C071_0', '12C081_0', '12C092_0', '12C102_0', '12C151_0', '12C161_0', '12C171_0', '12C181_0', '12C201_0', '12C211_0', '12C221_0', '12C251_0', '12C271_0', '12C281_0', '12C291_0', '12C301_0', '12C311_0', '12C321_0', '12C341_0', '12C371_0', '12C381_0', '12C391_0', '12C401_0', '12C421_0', '12C431_0', '12C441_0', '12C471_0', '12C481_0', '12C521_0', '12C551_0', '12C561_0', '12C601_0', '12C611_0', '12C621_0', '12C631_0', '12C641_0', '12C671_0', '12C681_0', '12C691_0', '12C711_0', '12C721_0', '12C731_0', '12C741_0', '12C771_0', '12P021_1', '12P031_1', '12P041_1', '12P051_1', '12P061_1', '12P091_1', '12P142_2', '12P161_1', '12P211_1', '12P221_1', '12P261_1', '12P391_1', '12P431_1', '12P441_1', '12P451_1', '12P471_1', '12P501_1', '12P541_1', '12P581_1', '12P641_1', '12P681_1', '12P761_1', '130C161_0', '130C221_0', '130C271_0', '130C311_0', '130C371_0', '130C391_0', '130C401_0', '130C441_0', '130C471_0', '130P451_1', '131C161_0', '131C221_0', '131C271_0', '131C311_0', '131C371_0', '131C391_0', '131C401_0', '131C441_0', '131C471_0', '131P451_1', '132C161_0', '132C221_0', '132C271_0', '132C311_0', '132C371_0', '132C391_0', '132C401_0', '132C441_0', '132C471_0', '132P451_1', '133C161_0', '133C221_0', '133C271_0', '133C311_0', '133C371_0', '133C391_0', '133C401_0', '133C441_0', '133C471_0', '133P451_1', '134C161_0', '134C221_0', '134C271_0', '134C311_0', '134C371_0', '134C391_0', '134C401_0', '134C441_0', '134C471_0', '134P451_1', '135C161_0', '135C221_0', '135C271_0', '135C311_0', '135C371_0', '135C391_0', '135C401_0', '135C441_0', '135C471_0', '135P451_1', '136C161_0', '136C221_0', '136C271_0', '136C311_0', '136C371_0', '136C391_0', '136C401_0', '136C441_0', '136C471_0', '136P451_1', '137C161_0', '137C221_0', '137C271_0', '137C311_0', '137C371_0', '137C391_0', '137C401_0', '137C441_0', '137C471_0', '137P451_1', '138C161_0', '138C221_0', '138C271_0', '138C311_0', '138C371_0', '138C391_0', '138C401_0', '138C441_0', '138C471_0', '138P451_1', '139C161_0', '139C221_0', '139C271_0', '139C311_0', '139C371_0', '139C391_0', '139C401_0', '139C441_0', '139C471_0', '139P451_1', '13C021_0', '13C042_0', '13C071_0', '13C081_0', '13C092_0', '13C102_0', '13C151_0', '13C161_0', '13C171_0', '13C181_0', '13C201_0', '13C211_0', '13C221_0', '13C251_0', '13C271_0', '13C281_0', '13C291_0', '13C301_0', '13C311_0', '13C321_0', '13C341_0', '13C371_0', '13C381_0', '13C391_0', '13C401_0', '13C421_0', '13C431_0', '13C441_0', '13C471_0', '13C481_0', '13C521_0', '13C551_0', '13C561_0', '13C601_0', '13C611_0', '13C621_0', '13C631_0', '13C641_0', '13C671_0', '13C681_0', '13C691_0', '13C711_0', '13C721_0', '13C731_0', '13C741_0', '13C771_0', '13P021_1', '13P041_1', '13P051_1', '13P061_1', '13P091_1', '13P142_2', '13P161_1', '13P211_1', '13P221_1', '13P261_1', '13P391_1', '13P431_1', '13P441_1', '13P451_1', '13P471_1', '13P541_1', '13P581_1', '13P641_1', '13P681_1', '13P761_1', '140C161_0', '140C221_0', '140C271_0', '140C311_0', '140C371_0', '140C391_0', '140C401_0', '140C441_0', '140C471_0', '140P451_1', '141C161_0', '141C221_0', '141C271_0', '141C311_0', '141C371_0', '141C391_0', '141C401_0', '141C441_0', '141C471_0', '141P451_1', '142C161_0', '142C221_0', '142C271_0', '142C311_0', '142C371_0', '142C391_0', '142C401_0', '142C441_0', '142C471_0', '142P451_1', '143C161_0', '143C221_0', '143C271_0', '143C311_0', '143C371_0', '143C391_0', '143C401_0', '143C471_0', '143P451_1', '144C161_0', '144C221_0', '144C271_0', '144C311_0', '144C371_0', '144C391_0', '144C401_0', '144C471_0', '144P451_1', '145C161_0', '145C221_0', '145C271_0', '145C311_0', '145C371_0', '145C391_0', '145C401_0', '145C471_0', '145P451_1', '146C161_0', '146C221_0', '146C271_0', '146C311_0', '146C371_0', '146C391_0', '146C401_0', '146C471_0', '146P451_1', '147C161_0', '147C221_0', '147C271_0', '147C311_0', '147C371_0', '147C391_0', '147C401_0', '147C471_0', '147P451_1', '148C161_0', '148C221_0', '148C271_0', '148C311_0', '148C371_0', '148C391_0', '148C401_0', '148C471_0', '148P451_1', '149C161_0', '149C221_0', '149C271_0', '149C311_0', '149C371_0', '149C391_0', '149C401_0', '149C471_0', '149P451_1', '14C021_0', '14C042_0', '14C071_0', '14C081_0', '14C092_0', '14C102_0', '14C151_0', '14C161_0', '14C171_0', '14C181_0', '14C201_0', '14C211_0', '14C221_0', '14C251_0', '14C271_0', '14C281_0', '14C291_0', '14C301_0', '14C311_0', '14C321_0', '14C341_0', '14C371_0', '14C381_0', '14C391_0', '14C401_0', '14C421_0', '14C431_0', '14C441_0', '14C471_0', '14C481_0', '14C521_0', '14C551_0', '14C561_0', '14C601_0', '14C611_0', '14C621_0', '14C631_0', '14C641_0', '14C671_0', '14C681_0', '14C691_0', '14C711_0', '14C721_0', '14C731_0', '14C741_0', '14C771_0', '14P021_1', '14P041_1', '14P051_1', '14P061_1', '14P091_1', '14P142_2', '14P161_1', '14P211_1', '14P221_1', '14P261_1', '14P391_1', '14P431_1', '14P441_1', '14P451_1', '14P471_1', '14P541_1', '14P581_1', '14P641_1', '14P681_1', '14P761_1', '150C161_0', '150C221_0', '150C271_0', '150C311_0', '150C371_0', '150C391_0', '150C401_0', '150C471_0', '150P451_1', '151C161_0', '151C221_0', '151C271_0', '151C311_0', '151C371_0', '151C391_0', '151C401_0', '151C471_0', '151P451_1', '152C161_0', '152C221_0', '152C271_0', '152C311_0', '152C371_0', '152C391_0', '152C401_0', '152C471_0', '152P451_1', '153C161_0', '153C221_0', '153C271_0', '153C311_0', '153C371_0', '153C391_0', '153C401_0', '153C471_0', '153P451_1', '154C161_0', '154C221_0', '154C271_0', '154C311_0', '154C371_0', '154C391_0', '154C401_0', '154C471_0', '154P451_1', '155C161_0', '155C221_0', '155C271_0', '155C311_0', '155C371_0', '155C391_0', '155C401_0', '155C471_0', '155P451_1', '156C161_0', '156C221_0', '156C271_0', '156C311_0', '156C371_0', '156C391_0', '156C401_0', '156C471_0', '157C161_0', '157C221_0', '157C271_0', '157C311_0', '157C371_0', '157C391_0', '157C401_0', '157C471_0', '158C161_0', '158C221_0', '158C271_0', '158C311_0', '158C371_0', '158C391_0', '158C401_0', '158C471_0', '159C161_0', '159C221_0', '159C271_0', '159C311_0', '159C371_0', '159C391_0', '159C401_0', '159C471_0', '15C021_0', '15C042_0', '15C071_0', '15C081_0', '15C092_0', '15C102_0', '15C151_0', '15C161_0', '15C171_0', '15C181_0', '15C201_0', '15C211_0', '15C221_0', '15C251_0', '15C271_0', '15C281_0', '15C291_0', '15C301_0', '15C311_0', '15C321_0', '15C341_0', '15C371_0', '15C381_0', '15C391_0', '15C401_0', '15C421_0', '15C431_0', '15C441_0', '15C471_0', '15C481_0', '15C521_0', '15C551_0', '15C561_0', '15C601_0', '15C611_0', '15C621_0', '15C631_0', '15C641_0', '15C671_0', '15C681_0', '15C691_0', '15C711_0', '15C721_0', '15C731_0', '15C741_0', '15C771_0', '15P021_1', '15P041_1', '15P051_1', '15P061_1', '15P091_1', '15P142_2', '15P161_1', '15P211_1', '15P221_1', '15P261_1', '15P391_1', '15P431_1', '15P441_1', '15P451_1', '15P471_1', '15P541_1', '15P581_1', '15P641_1', '15P681_1', '15P761_1', '160C161_0', '160C221_0', '160C271_0', '160C311_0', '160C371_0', '160C391_0', '160C401_0', '160C471_0', '161C161_0', '161C221_0', '161C271_0', '161C311_0', '161C371_0', '161C391_0', '161C401_0', '161C471_0', '162C221_0', '162C271_0', '162C311_0', '162C371_0', '162C391_0', '162C401_0', '162C471_0', '163C221_0', '163C271_0', '163C311_0', '163C371_0', '163C391_0', '163C401_0', '163C471_0', '164C221_0', '164C271_0', '164C311_0', '164C371_0', '164C391_0', '164C401_0', '164C471_0', '165C221_0', '165C271_0', '165C311_0', '165C371_0', '165C391_0', '165C401_0', '165C471_0', '166C221_0', '166C271_0', '166C311_0', '166C371_0', '166C391_0', '166C401_0', '166C471_0', '167C221_0', '167C271_0', '167C311_0', '167C371_0', '167C391_0', '167C401_0', '167C471_0', '168C221_0', '168C271_0', '168C311_0', '168C371_0', '168C391_0', '168C401_0', '168C471_0', '169C221_0', '169C271_0', '169C311_0', '169C371_0', '169C391_0', '169C401_0', '169C471_0', '16C021_0', '16C042_0', '16C071_0', '16C081_0', '16C092_0', '16C102_0', '16C151_0', '16C161_0', '16C171_0', '16C181_0', '16C201_0', '16C211_0', '16C221_0', '16C251_0', '16C271_0', '16C281_0', '16C291_0', '16C301_0', '16C311_0', '16C321_0', '16C341_0', '16C371_0', '16C381_0', '16C391_0', '16C401_0', '16C421_0', '16C431_0', '16C441_0', '16C471_0', '16C481_0', '16C521_0', '16C551_0', '16C561_0', '16C601_0', '16C611_0', '16C621_0', '16C631_0', '16C641_0', '16C671_0', '16C681_0', '16C691_0', '16C711_0', '16C721_0', '16C731_0', '16C741_0', '16C771_0', '16P021_1', '16P041_1', '16P051_1', '16P061_1', '16P091_1', '16P142_2', '16P161_1', '16P211_1', '16P221_1', '16P261_1', '16P391_1', '16P431_1', '16P441_1', '16P451_1', '16P471_1', '16P541_1', '16P581_1', '16P641_1', '16P681_1', '16P761_1', '170C221_0', '170C271_0', '170C311_0', '170C371_0', '170C391_0', '170C401_0', '170C471_0', '171C221_0', '171C271_0', '171C311_0', '171C371_0', '171C391_0', '171C401_0', '171C471_0', '172C221_0', '172C271_0', '172C311_0', '172C371_0', '172C391_0', '172C401_0', '172C471_0', '173C221_0', '173C271_0', '173C311_0', '173C371_0', '173C391_0', '173C401_0', '173C471_0', '174C221_0', '174C271_0', '174C311_0', '174C371_0', '174C391_0', '174C401_0', '174C471_0', '175C221_0', '175C271_0', '175C311_0', '175C371_0', '175C391_0', '175C401_0', '175C471_0', '176C221_0', '176C271_0', '176C311_0', '176C371_0', '176C391_0', '176C401_0', '176C471_0', '177C221_0', '177C271_0', '177C311_0', '177C371_0', '177C391_0', '177C401_0', '177C471_0', '178C221_0', '178C271_0', '178C311_0', '178C371_0', '178C391_0', '178C401_0', '178C471_0', '179C221_0', '179C271_0', '179C311_0', '179C371_0', '179C391_0', '179C401_0', '179C471_0', '17C021_0', '17C042_0', '17C071_0', '17C081_0', '17C092_0', '17C102_0', '17C151_0', '17C161_0', '17C171_0', '17C201_0', '17C211_0', '17C221_0', '17C251_0', '17C271_0', '17C281_0', '17C291_0', '17C301_0', '17C311_0', '17C321_0', '17C341_0', '17C371_0', '17C381_0', '17C391_0', '17C401_0', '17C421_0', '17C431_0', '17C441_0', '17C471_0', '17C481_0', '17C521_0', '17C551_0', '17C561_0', '17C601_0', '17C611_0', '17C621_0', '17C631_0', '17C641_0', '17C671_0', '17C681_0', '17C691_0', '17C711_0', '17C721_0', '17C731_0', '17C741_0', '17C771_0', '17P021_1', '17P041_1', '17P051_1', '17P061_1', '17P091_1', '17P142_2', '17P161_1', '17P211_1', '17P221_1', '17P261_1', '17P391_1', '17P431_1', '17P441_1', '17P451_1', '17P471_1', '17P541_1', '17P581_1', '17P641_1', '17P681_1', '17P761_1', '180C221_0', '180C271_0', '180C311_0', '180C371_0', '180C391_0', '180C401_0', '180C471_0', '181C221_0', '181C271_0', '181C311_0', '181C371_0', '181C391_0', '181C401_0', '181C471_0', '182C221_0', '182C271_0', '182C311_0', '182C371_0', '182C391_0', '182C401_0', '182C471_0', '183C221_0', '183C271_0', '183C311_0', '183C371_0', '183C391_0', '183C401_0', '183C471_0', '184C221_0', '184C311_0', '184C371_0', '184C391_0', '184C401_0', '184C471_0', '185C221_0', '185C311_0', '185C371_0', '185C391_0', '185C401_0', '185C471_0', '186C221_0', '186C311_0', '186C371_0', '186C391_0', '186C401_0', '186C471_0', '187C221_0', '187C311_0', '187C371_0', '187C391_0', '187C401_0', '187C471_0', '188C221_0', '188C311_0', '188C371_0', '188C391_0', '188C401_0', '188C471_0', '189C221_0', '189C311_0', '189C371_0', '189C391_0', '189C401_0', '189C471_0', '18C021_0', '18C042_0', '18C071_0', '18C081_0', '18C092_0', '18C102_0', '18C151_0', '18C161_0', '18C171_0', '18C201_0', '18C211_0', '18C221_0', '18C251_0', '18C271_0', '18C281_0', '18C291_0', '18C301_0', '18C311_0', '18C321_0', '18C341_0', '18C371_0', '18C381_0', '18C391_0', '18C401_0', '18C421_0', '18C431_0', '18C441_0', '18C471_0', '18C481_0', '18C521_0', '18C551_0', '18C561_0', '18C601_0', '18C611_0', '18C621_0', '18C631_0', '18C641_0', '18C671_0', '18C681_0', '18C691_0', '18C711_0', '18C721_0', '18C731_0', '18C741_0', '18C771_0', '18P021_1', '18P041_1', '18P051_1', '18P061_1', '18P091_1', '18P142_2', '18P161_1', '18P211_1', '18P221_1', '18P261_1', '18P391_1', '18P431_1', '18P441_1', '18P451_1', '18P471_1', '18P541_1', '18P581_1', '18P641_1', '18P681_1', '18P761_1', '190C221_0', '190C311_0', '190C371_0', '190C391_0', '190C401_0', '190C471_0', '191C221_0', '191C311_0', '191C371_0', '191C391_0', '191C401_0', '191C471_0', '192C221_0', '192C311_0', '192C371_0', '192C391_0', '192C401_0', '192C471_0', '193C221_0', '193C311_0', '193C371_0', '193C391_0', '193C401_0', '193C471_0', '194C221_0', '194C311_0', '194C371_0', '194C391_0', '194C401_0', '195C221_0', '195C311_0', '195C371_0', '195C391_0', '195C401_0', '196C221_0', '196C311_0', '196C371_0', '196C391_0', '196C401_0', '197C221_0', '197C311_0', '197C371_0', '197C391_0', '197C401_0', '198C221_0', '198C311_0', '198C371_0', '198C391_0', '198C401_0', '199C221_0', '199C311_0', '199C371_0', '199C391_0', '199C401_0', '19C021_0', '19C042_0', '19C071_0', '19C081_0', '19C092_0', '19C102_0', '19C151_0', '19C161_0', '19C171_0', '19C201_0', '19C211_0', '19C221_0', '19C251_0', '19C271_0', '19C281_0', '19C291_0', '19C301_0', '19C311_0', '19C321_0', '19C341_0', '19C371_0', '19C381_0', '19C391_0', '19C401_0', '19C421_0', '19C431_0', '19C441_0', '19C471_0', '19C481_0', '19C521_0', '19C551_0', '19C561_0', '19C601_0', '19C611_0', '19C621_0', '19C631_0', '19C641_0', '19C671_0', '19C681_0', '19C691_0', '19C711_0', '19C721_0', '19C731_0', '19C741_0', '19P021_1', '19P041_1', '19P051_1', '19P061_1', '19P142_2', '19P161_1', '19P211_1', '19P221_1', '19P261_1', '19P391_1', '19P431_1', '19P441_1', '19P451_1', '19P471_1', '19P541_1', '19P581_1', '19P641_1', '19P681_1', '19P761_1', '1C021_0', '1C042_0', '1C071_0', '1C081_0', '1C092_0', '1C102_0', '1C132_0', '1C151_0', '1C161_0', '1C171_0', '1C181_0', '1C201_0', '1C211_0', '1C221_0', '1C251_0', '1C271_0', '1C281_0', '1C291_0', '1C301_0', '1C311_0', '1C321_0', '1C341_0', '1C361_0', '1C371_0', '1C381_0', '1C391_0', '1C401_0', '1C411_0', '1C421_0', '1C431_0', '1C441_0', '1C471_0', '1C481_0', '1C521_0', '1C551_0', '1C561_0', '1C601_0', '1C611_0', '1C621_0', '1C631_0', '1C641_0', '1C651_0', '1C671_0', '1C681_0', '1C691_0', '1C711_0', '1C721_0', '1C731_0', '1C741_0', '1C771_0', '1P021_1', '1P031_1', '1P041_1', '1P051_1', '1P061_1', '1P091_1', '1P112_2', '1P131_1', '1P142_2', '1P161_1', '1P211_1', '1P221_1', '1P261_1', '1P391_1', '1P411_1', '1P431_1', '1P441_1', '1P451_1', '1P461_1', '1P471_1', '1P501_1', '1P541_1', '1P581_1', '1P641_1', '1P681_1', '1P691_1', '1P761_1', '200C221_0', '200C311_0', '200C371_0', '200C391_0', '200C401_0', '201C221_0', '201C311_0', '201C371_0', '201C391_0', '201C401_0', '202C221_0', '202C311_0', '202C371_0', '202C391_0', '202C401_0', '203C221_0', '203C311_0', '203C371_0', '203C391_0', '203C401_0', '204C221_0', '204C311_0', '204C371_0', '204C391_0', '204C401_0', '205C221_0', '205C311_0', '205C371_0', '205C391_0', '205C401_0', '206C221_0', '206C311_0', '206C371_0', '206C391_0', '206C401_0', '207C221_0', '207C311_0', '207C371_0', '207C391_0', '207C401_0', '208C221_0', '208C311_0', '208C371_0', '208C391_0', '208C401_0', '209C221_0', '209C311_0', '209C371_0', '209C391_0', '209C401_0', '20C021_0', '20C042_0', '20C071_0', '20C081_0', '20C092_0', '20C102_0', '20C151_0', '20C161_0', '20C171_0', '20C201_0', '20C211_0', '20C221_0', '20C251_0', '20C271_0', '20C281_0', '20C291_0', '20C301_0', '20C311_0', '20C321_0', '20C341_0', '20C371_0', '20C381_0', '20C391_0', '20C401_0', '20C421_0', '20C431_0', '20C441_0', '20C471_0', '20C481_0', '20C521_0', '20C551_0', '20C561_0', '20C601_0', '20C611_0', '20C621_0', '20C631_0', '20C641_0', '20C671_0', '20C681_0', '20C691_0', '20C711_0', '20C721_0', '20C731_0', '20C741_0', '20P021_1', '20P041_1', '20P051_1', '20P061_1', '20P142_2', '20P161_1', '20P211_1', '20P221_1', '20P261_1', '20P431_1', '20P441_1', '20P451_1', '20P471_1', '20P541_1', '20P581_1', '20P641_1', '20P681_1', '20P761_1', '210C221_0', '210C311_0', '210C371_0', '210C391_0', '210C401_0', '211C221_0', '211C311_0', '211C371_0', '211C391_0', '211C401_0', '212C221_0', '212C311_0', '212C371_0', '212C391_0', '212C401_0', '213C221_0', '213C311_0', '213C371_0', '213C391_0', '213C401_0', '214C311_0', '214C371_0', '214C391_0', '214C401_0', '215C311_0', '215C371_0', '215C391_0', '215C401_0', '216C311_0', '216C371_0', '216C391_0', '216C401_0', '217C311_0', '217C371_0', '217C391_0', '217C401_0', '218C311_0', '218C371_0', '218C391_0', '218C401_0', '219C311_0', '219C391_0', '219C401_0', '21C021_0', '21C042_0', '21C071_0', '21C081_0', '21C092_0', '21C102_0', '21C151_0', '21C161_0', '21C171_0', '21C201_0', '21C211_0', '21C221_0', '21C251_0', '21C271_0', '21C281_0', '21C291_0', '21C301_0', '21C311_0', '21C321_0', '21C341_0', '21C371_0', '21C381_0', '21C391_0', '21C401_0', '21C421_0', '21C431_0', '21C441_0', '21C471_0', '21C481_0', '21C521_0', '21C551_0', '21C561_0', '21C601_0', '21C611_0', '21C621_0', '21C631_0', '21C641_0', '21C671_0', '21C681_0', '21C691_0', '21C711_0', '21C721_0', '21C731_0', '21C741_0', '21P021_1', '21P041_1', '21P051_1', '21P061_1', '21P142_2', '21P161_1', '21P211_1', '21P221_1', '21P261_1', '21P441_1', '21P451_1', '21P471_1', '21P541_1', '21P581_1', '21P641_1', '21P681_1', '21P761_1', '220C311_0', '220C391_0', '220C401_0', '221C311_0', '221C391_0', '221C401_0', '222C311_0', '222C391_0', '222C401_0', '223C311_0', '223C391_0', '223C401_0', '224C311_0', '224C391_0', '224C401_0', '225C311_0', '225C391_0', '225C401_0', '226C311_0', '226C391_0', '226C401_0', '227C311_0', '227C391_0', '227C401_0', '228C311_0', '228C391_0', '228C401_0', '229C311_0', '229C391_0', '229C401_0', '22C021_0', '22C042_0', '22C071_0', '22C081_0', '22C092_0', '22C151_0', '22C161_0', '22C171_0', '22C201_0', '22C211_0', '22C221_0', '22C251_0', '22C271_0', '22C281_0', '22C291_0', '22C301_0', '22C311_0', '22C321_0', '22C341_0', '22C371_0', '22C381_0', '22C391_0', '22C401_0', '22C421_0', '22C431_0', '22C441_0', '22C471_0', '22C481_0', '22C521_0', '22C551_0', '22C561_0', '22C601_0', '22C611_0', '22C621_0', '22C631_0', '22C641_0', '22C671_0', '22C681_0', '22C691_0', '22C711_0', '22C721_0', '22C731_0', '22C741_0', '22P021_1', '22P041_1', '22P051_1', '22P061_1', '22P142_2', '22P161_1', '22P211_1', '22P221_1', '22P261_1', '22P441_1', '22P451_1', '22P471_1', '22P541_1', '22P581_1', '22P641_1', '22P761_1', '230C311_0', '230C391_0', '230C401_0', '231C311_0', '231C391_0', '231C401_0', '232C311_0', '232C391_0', '232C401_0', '233C311_0', '233C391_0', '233C401_0', '234C311_0', '234C391_0', '234C401_0', '235C311_0', '235C391_0', '235C401_0', '236C311_0', '236C391_0', '236C401_0', '237C391_0', '237C401_0', '238C391_0', '238C401_0', '239C391_0', '239C401_0', '23C021_0', '23C042_0', '23C071_0', '23C081_0', '23C092_0', '23C151_0', '23C161_0', '23C171_0', '23C201_0', '23C211_0', '23C221_0', '23C251_0', '23C271_0', '23C281_0', '23C291_0', '23C301_0', '23C311_0', '23C321_0', '23C341_0', '23C371_0', '23C381_0', '23C391_0', '23C401_0', '23C421_0', '23C431_0', '23C441_0', '23C471_0', '23C481_0', '23C521_0', '23C551_0', '23C561_0', '23C601_0', '23C611_0', '23C621_0', '23C631_0', '23C641_0', '23C671_0', '23C681_0', '23C691_0', '23C711_0', '23C721_0', '23C731_0', '23C741_0', '23P021_1', '23P041_1', '23P051_1', '23P061_1', '23P142_2', '23P161_1', '23P211_1', '23P221_1', '23P261_1', '23P441_1', '23P451_1', '23P471_1', '23P541_1', '23P581_1', '23P641_1', '23P761_1', '240C391_0', '240C401_0', '241C391_0', '241C401_0', '242C391_0', '242C401_0', '243C391_0', '243C401_0', '244C391_0', '244C401_0', '245C391_0', '245C401_0', '246C391_0', '246C401_0', '247C391_0', '247C401_0', '248C391_0', '248C401_0', '249C391_0', '249C401_0', '24C021_0', '24C042_0', '24C071_0', '24C081_0', '24C092_0', '24C161_0', '24C171_0', '24C201_0', '24C211_0', '24C221_0', '24C251_0', '24C271_0', '24C281_0', '24C291_0', '24C301_0', '24C311_0', '24C321_0', '24C341_0', '24C371_0', '24C381_0', '24C391_0', '24C401_0', '24C421_0', '24C431_0', '24C441_0', '24C471_0', '24C481_0', '24C521_0', '24C551_0', '24C561_0', '24C601_0', '24C611_0', '24C621_0', '24C641_0', '24C671_0', '24C681_0', '24C691_0', '24C711_0', '24C721_0', '24C731_0', '24C741_0', '24P021_1', '24P041_1', '24P051_1', '24P061_1', '24P142_2', '24P161_1', '24P211_1', '24P221_1', '24P261_1', '24P441_1', '24P451_1', '24P471_1', '24P541_1', '24P581_1', '24P641_1', '24P761_1', '250C391_0', '250C401_0', '251C391_0', '251C401_0', '252C391_0', '252C401_0', '253C391_0', '253C401_0', '254C391_0', '254C401_0', '255C391_0', '255C401_0', '256C391_0', '256C401_0', '257C391_0', '257C401_0', '258C391_0', '258C401_0', '259C391_0', '259C401_0', '25C021_0', '25C042_0', '25C071_0', '25C081_0', '25C092_0', '25C161_0', '25C171_0', '25C201_0', '25C211_0', '25C221_0', '25C251_0', '25C271_0', '25C281_0', '25C291_0', '25C301_0', '25C311_0', '25C321_0', '25C341_0', '25C371_0', '25C381_0', '25C391_0', '25C401_0', '25C421_0', '25C431_0', '25C441_0', '25C471_0', '25C481_0', '25C521_0', '25C551_0', '25C561_0', '25C601_0', '25C611_0', '25C621_0', '25C671_0', '25C681_0', '25C691_0', '25C711_0', '25C721_0', '25C731_0', '25C741_0', '25P021_1', '25P041_1', '25P051_1', '25P061_1', '25P142_2', '25P161_1', '25P211_1', '25P221_1', '25P261_1', '25P441_1', '25P451_1', '25P471_1', '25P541_1', '25P581_1', '25P641_1', '25P761_1', '260C391_0', '260C401_0', '261C391_0', '261C401_0', '262C391_0', '262C401_0', '263C391_0', '263C401_0', '264C391_0', '264C401_0', '265C391_0', '265C401_0', '266C391_0', '266C401_0', '267C391_0', '267C401_0', '268C391_0', '268C401_0', '269C391_0', '269C401_0', '26C021_0', '26C042_0', '26C071_0', '26C081_0', '26C092_0', '26C161_0', '26C171_0', '26C211_0', '26C221_0', '26C251_0', '26C271_0', '26C281_0', '26C291_0', '26C301_0', '26C311_0', '26C321_0', '26C371_0', '26C381_0', '26C391_0', '26C401_0', '26C421_0', '26C431_0', '26C441_0', '26C471_0', '26C521_0', '26C551_0', '26C561_0', '26C601_0', '26C611_0', '26C621_0', '26C671_0', '26C681_0', '26C691_0', '26C711_0', '26C721_0', '26C731_0', '26C741_0', '26P021_1', '26P041_1', '26P051_1', '26P061_1', '26P142_2', '26P161_1', '26P211_1', '26P221_1', '26P261_1', '26P441_1', '26P451_1', '26P471_1', '26P541_1', '26P581_1', '26P641_1', '26P761_1', '270C391_0', '270C401_0', '271C391_0', '271C401_0', '272C391_0', '272C401_0', '273C391_0', '273C401_0', '274C391_0', '274C401_0', '275C391_0', '275C401_0', '276C391_0', '276C401_0', '277C391_0', '277C401_0', '278C391_0', '278C401_0', '279C391_0', '279C401_0', '27C042_0', '27C071_0', '27C081_0', '27C092_0', '27C161_0', '27C171_0', '27C211_0', '27C221_0', '27C251_0', '27C271_0', '27C281_0', '27C291_0', '27C301_0', '27C311_0', '27C321_0', '27C371_0', '27C381_0', '27C391_0', '27C401_0', '27C421_0', '27C431_0', '27C441_0', '27C471_0', '27C521_0', '27C551_0', '27C561_0', '27C601_0', '27C611_0', '27C621_0', '27C671_0', '27C681_0', '27C691_0', '27C711_0', '27C721_0', '27C731_0', '27C741_0', '27P021_1', '27P041_1', '27P051_1', '27P061_1', '27P142_2', '27P161_1', '27P211_1', '27P221_1', '27P261_1', '27P441_1', '27P451_1', '27P471_1', '27P541_1', '27P581_1', '27P641_1', '27P761_1', '280C391_0', '280C401_0', '281C391_0', '281C401_0', '282C391_0', '282C401_0', '283C391_0', '283C401_0', '284C391_0', '284C401_0', '285C391_0', '285C401_0', '286C391_0', '286C401_0', '287C391_0', '287C401_0', '288C391_0', '288C401_0', '289C391_0', '289C401_0', '28C042_0', '28C071_0', '28C081_0', '28C092_0', '28C161_0', '28C171_0', '28C211_0', '28C221_0', '28C251_0', '28C271_0', '28C281_0', '28C291_0', '28C301_0', '28C311_0', '28C321_0', '28C371_0', '28C381_0', '28C391_0', '28C401_0', '28C421_0', '28C431_0', '28C441_0', '28C471_0', '28C521_0', '28C551_0', '28C561_0', '28C601_0', '28C611_0', '28C621_0', '28C671_0', '28C681_0', '28C691_0', '28C711_0', '28C721_0', '28C731_0', '28C741_0', '28P021_1', '28P041_1', '28P051_1', '28P061_1', '28P142_2', '28P161_1', '28P211_1', '28P221_1', '28P261_1', '28P441_1', '28P451_1', '28P471_1', '28P541_1', '28P581_1', '28P641_1', '28P761_1', '290C391_0', '290C401_0', '291C391_0', '291C401_0', '292C391_0', '292C401_0', '293C391_0', '293C401_0', '294C391_0', '294C401_0', '295C391_0', '295C401_0', '296C391_0', '296C401_0', '297C391_0', '297C401_0', '298C391_0', '298C401_0', '299C391_0', '299C401_0', '29C071_0', '29C081_0', '29C092_0', '29C161_0', '29C171_0', '29C211_0', '29C221_0', '29C251_0', '29C271_0', '29C291_0', '29C301_0', '29C311_0', '29C321_0', '29C371_0', '29C381_0', '29C391_0', '29C401_0', '29C421_0', '29C431_0', '29C441_0', '29C471_0', '29C521_0', '29C551_0', '29C561_0', '29C601_0', '29C611_0', '29C621_0', '29C671_0', '29C681_0', '29C711_0', '29C721_0', '29C731_0', '29C741_0', '29P021_1', '29P041_1', '29P051_1', '29P061_1', '29P142_2', '29P161_1', '29P211_1', '29P221_1', '29P261_1', '29P441_1', '29P451_1', '29P471_1', '29P541_1', '29P581_1', '29P641_1', '29P761_1', '2C021_0', '2C042_0', '2C071_0', '2C081_0', '2C092_0', '2C102_0', '2C132_0', '2C151_0', '2C161_0', '2C171_0', '2C181_0', '2C201_0', '2C211_0', '2C221_0', '2C251_0', '2C271_0', '2C281_0', '2C291_0', '2C301_0', '2C311_0', '2C321_0', '2C341_0', '2C361_0', '2C371_0', '2C381_0', '2C391_0', '2C401_0', '2C411_0', '2C421_0', '2C431_0', '2C441_0', '2C471_0', '2C481_0', '2C521_0', '2C551_0', '2C561_0', '2C601_0', '2C611_0', '2C621_0', '2C631_0', '2C641_0', '2C651_0', '2C671_0', '2C681_0', '2C691_0', '2C711_0', '2C721_0', '2C731_0', '2C741_0', '2C771_0', '2P021_1', '2P031_1', '2P041_1', '2P051_1', '2P061_1', '2P091_1', '2P112_2', '2P131_1', '2P142_2', '2P161_1', '2P211_1', '2P221_1', '2P261_1', '2P391_1', '2P411_1', '2P431_1', '2P441_1', '2P451_1', '2P461_1', '2P471_1', '2P501_1', '2P541_1', '2P581_1', '2P641_1', '2P681_1', '2P691_1', '2P761_1', '300C391_0', '300C401_0', '301C391_0', '301C401_0', '302C391_0', '302C401_0', '303C391_0', '303C401_0', '304C391_0', '304C401_0', '305C391_0', '305C401_0', '306C391_0', '306C401_0', '307C391_0', '307C401_0', '308C391_0', '308C401_0', '309C391_0', '309C401_0', '30C071_0', '30C081_0', '30C092_0', '30C161_0', '30C171_0', '30C211_0', '30C221_0', '30C251_0', '30C271_0', '30C291_0', '30C301_0', '30C311_0', '30C321_0', '30C371_0', '30C381_0', '30C391_0', '30C401_0', '30C421_0', '30C431_0', '30C441_0', '30C471_0', '30C521_0', '30C551_0', '30C561_0', '30C601_0', '30C611_0', '30C621_0', '30C671_0', '30C681_0', '30C711_0', '30C721_0', '30C731_0', '30C741_0', '30P021_1', '30P041_1', '30P051_1', '30P061_1', '30P142_2', '30P161_1', '30P211_1', '30P221_1', '30P261_1', '30P441_1', '30P451_1', '30P471_1', '30P541_1', '30P581_1', '30P641_1', '30P761_1', '310C391_0', '310C401_0', '311C391_0', '311C401_0', '312C391_0', '312C401_0', '313C391_0', '313C401_0', '314C391_0', '314C401_0', '315C391_0', '315C401_0', '316C391_0', '316C401_0', '317C391_0', '317C401_0', '318C391_0', '318C401_0', '319C391_0', '319C401_0', '31C071_0', '31C081_0', '31C092_0', '31C161_0', '31C211_0', '31C221_0', '31C251_0', '31C271_0', '31C291_0', '31C301_0', '31C311_0', '31C321_0', '31C371_0', '31C381_0', '31C391_0', '31C401_0', '31C421_0', '31C431_0', '31C441_0', '31C471_0', '31C521_0', '31C551_0', '31C561_0', '31C601_0', '31C611_0', '31C621_0', '31C671_0', '31C681_0', '31C711_0', '31C721_0', '31C731_0', '31C741_0', '31P021_1', '31P041_1', '31P051_1', '31P061_1', '31P142_2', '31P161_1', '31P211_1', '31P221_1', '31P261_1', '31P451_1', '31P471_1', '31P541_1', '31P581_1', '31P641_1', '31P761_1', '320C391_0', '320C401_0', '321C391_0', '321C401_0', '322C391_0', '322C401_0', '323C391_0', '323C401_0', '324C391_0', '324C401_0', '325C391_0', '325C401_0', '326C391_0', '326C401_0', '327C391_0', '327C401_0', '328C391_0', '328C401_0', '329C391_0', '329C401_0', '32C071_0', '32C081_0', '32C092_0', '32C161_0', '32C211_0', '32C221_0', '32C251_0', '32C271_0', '32C291_0', '32C301_0', '32C311_0', '32C321_0', '32C371_0', '32C381_0', '32C391_0', '32C401_0', '32C421_0', '32C431_0', '32C441_0', '32C471_0', '32C521_0', '32C551_0', '32C561_0', '32C601_0', '32C611_0', '32C621_0', '32C671_0', '32C681_0', '32C711_0', '32C721_0', '32C731_0', '32C741_0', '32P021_1', '32P041_1', '32P051_1', '32P061_1', '32P142_2', '32P161_1', '32P211_1', '32P221_1', '32P261_1', '32P451_1', '32P471_1', '32P541_1', '32P581_1', '32P641_1', '32P761_1', '330C391_0', '330C401_0', '331C391_0', '331C401_0', '332C391_0', '332C401_0', '333C391_0', '333C401_0', '334C391_0', '334C401_0', '335C391_0', '335C401_0', '336C391_0', '336C401_0', '337C391_0', '338C391_0', '339C391_0', '33C071_0', '33C081_0', '33C092_0', '33C161_0', '33C211_0', '33C221_0', '33C251_0', '33C271_0', '33C291_0', '33C301_0', '33C311_0', '33C321_0', '33C371_0', '33C381_0', '33C391_0', '33C401_0', '33C421_0', '33C431_0', '33C441_0', '33C471_0', '33C521_0', '33C551_0', '33C561_0', '33C601_0', '33C611_0', '33C621_0', '33C671_0', '33C681_0', '33C711_0', '33C721_0', '33C731_0', '33C741_0', '33P021_1', '33P041_1', '33P051_1', '33P061_1', '33P161_1', '33P211_1', '33P221_1', '33P261_1', '33P451_1', '33P471_1', '33P541_1', '33P581_1', '33P641_1', '33P761_1', '340C391_0', '341C391_0', '342C391_0', '343C391_0', '344C391_0', '345C391_0', '346C391_0', '347C391_0', '348C391_0', '34C071_0', '34C081_0', '34C092_0', '34C161_0', '34C211_0', '34C221_0', '34C251_0', '34C271_0', '34C291_0', '34C301_0', '34C311_0', '34C321_0', '34C371_0', '34C381_0', '34C391_0', '34C401_0', '34C421_0', '34C431_0', '34C441_0', '34C471_0', '34C521_0', '34C551_0', '34C561_0', '34C601_0', '34C611_0', '34C621_0', '34C671_0', '34C681_0', '34C711_0', '34C721_0', '34C731_0', '34C741_0', '34P021_1', '34P041_1', '34P051_1', '34P061_1', '34P161_1', '34P211_1', '34P221_1', '34P261_1', '34P451_1', '34P471_1', '34P541_1', '34P581_1', '34P641_1', '34P761_1', '35C071_0', '35C081_0', '35C092_0', '35C161_0', '35C211_0', '35C221_0', '35C251_0', '35C271_0', '35C291_0', '35C301_0', '35C311_0', '35C321_0', '35C371_0', '35C381_0', '35C391_0', '35C401_0', '35C421_0', '35C431_0', '35C441_0', '35C471_0', '35C521_0', '35C551_0', '35C561_0', '35C601_0', '35C611_0', '35C621_0', '35C671_0', '35C681_0', '35C711_0', '35C721_0', '35C731_0', '35C741_0', '35P021_1', '35P041_1', '35P051_1', '35P061_1', '35P161_1', '35P211_1', '35P221_1', '35P261_1', '35P451_1', '35P471_1', '35P541_1', '35P581_1', '35P641_1', '35P761_1', '36C071_0', '36C081_0', '36C092_0', '36C161_0', '36C211_0', '36C221_0', '36C251_0', '36C271_0', '36C291_0', '36C301_0', '36C311_0', '36C321_0', '36C371_0', '36C381_0', '36C391_0', '36C401_0', '36C421_0', '36C431_0', '36C441_0', '36C471_0', '36C521_0', '36C551_0', '36C561_0', '36C601_0', '36C611_0', '36C621_0', '36C671_0', '36C681_0', '36C711_0', '36C721_0', '36C731_0', '36C741_0', '36P021_1', '36P041_1', '36P051_1', '36P061_1', '36P161_1', '36P211_1', '36P221_1', '36P261_1', '36P451_1', '36P471_1', '36P541_1', '36P581_1', '36P641_1', '36P761_1', '37C071_0', '37C081_0', '37C092_0', '37C161_0', '37C211_0', '37C221_0', '37C251_0', '37C271_0', '37C291_0', '37C301_0', '37C311_0', '37C321_0', '37C371_0', '37C381_0', '37C391_0', '37C401_0', '37C421_0', '37C431_0', '37C441_0', '37C471_0', '37C521_0', '37C551_0', '37C561_0', '37C601_0', '37C611_0', '37C621_0', '37C671_0', '37C681_0', '37C711_0', '37C721_0', '37C731_0', '37C741_0', '37P021_1', '37P041_1', '37P051_1', '37P061_1', '37P161_1', '37P211_1', '37P221_1', '37P261_1', '37P451_1', '37P471_1', '37P541_1', '37P581_1', '37P641_1', '37P761_1', '38C071_0', '38C081_0', '38C092_0', '38C161_0', '38C211_0', '38C221_0', '38C251_0', '38C271_0', '38C291_0', '38C301_0', '38C311_0', '38C321_0', '38C371_0', '38C381_0', '38C391_0', '38C401_0', '38C421_0', '38C431_0', '38C441_0', '38C471_0', '38C521_0', '38C551_0', '38C561_0', '38C601_0', '38C611_0', '38C621_0', '38C671_0', '38C681_0', '38C711_0', '38C721_0', '38C731_0', '38C741_0', '38P021_1', '38P041_1', '38P051_1', '38P061_1', '38P161_1', '38P211_1', '38P221_1', '38P261_1', '38P451_1', '38P471_1', '38P541_1', '38P581_1', '38P641_1', '38P761_1', '39C071_0', '39C081_0', '39C092_0', '39C161_0', '39C211_0', '39C221_0', '39C251_0', '39C271_0', '39C291_0', '39C301_0', '39C311_0', '39C321_0', '39C371_0', '39C381_0', '39C391_0', '39C401_0', '39C421_0', '39C431_0', '39C441_0', '39C471_0', '39C521_0', '39C551_0', '39C561_0', '39C601_0', '39C611_0', '39C621_0', '39C671_0', '39C681_0', '39C711_0', '39C721_0', '39C731_0', '39C741_0', '39P021_1', '39P041_1', '39P051_1', '39P061_1', '39P161_1', '39P211_1', '39P221_1', '39P261_1', '39P451_1', '39P471_1', '39P581_1', '39P641_1', '39P761_1', '3C021_0', '3C042_0', '3C071_0', '3C081_0', '3C092_0', '3C102_0', '3C132_0', '3C151_0', '3C161_0', '3C171_0', '3C181_0', '3C201_0', '3C211_0', '3C221_0', '3C251_0', '3C271_0', '3C281_0', '3C291_0', '3C301_0', '3C311_0', '3C321_0', '3C341_0', '3C361_0', '3C371_0', '3C381_0', '3C391_0', '3C401_0', '3C411_0', '3C421_0', '3C431_0', '3C441_0', '3C471_0', '3C481_0', '3C521_0', '3C551_0', '3C561_0', '3C601_0', '3C611_0', '3C621_0', '3C631_0', '3C641_0', '3C651_0', '3C671_0', '3C681_0', '3C691_0', '3C711_0', '3C721_0', '3C731_0', '3C741_0', '3C771_0', '3P021_1', '3P031_1', '3P041_1', '3P051_1', '3P061_1', '3P091_1', '3P112_2', '3P131_1', '3P142_2', '3P161_1', '3P211_1', '3P221_1', '3P261_1', '3P391_1', '3P411_1', '3P431_1', '3P441_1', '3P451_1', '3P461_1', '3P471_1', '3P501_1', '3P541_1', '3P581_1', '3P641_1', '3P681_1', '3P761_1', '40C071_0', '40C081_0', '40C092_0', '40C161_0', '40C211_0', '40C221_0', '40C251_0', '40C271_0', '40C291_0', '40C301_0', '40C311_0', '40C321_0', '40C371_0', '40C381_0', '40C391_0', '40C401_0', '40C421_0', '40C431_0', '40C441_0', '40C471_0', '40C521_0', '40C551_0', '40C601_0', '40C611_0', '40C621_0', '40C671_0', '40C681_0', '40C711_0', '40C721_0', '40C731_0', '40C741_0', '40P021_1', '40P041_1', '40P051_1', '40P061_1', '40P161_1', '40P211_1', '40P221_1', '40P261_1', '40P451_1', '40P471_1', '40P581_1', '40P641_1', '40P761_1', '41C071_0', '41C092_0', '41C161_0', '41C211_0', '41C221_0', '41C251_0', '41C271_0', '41C291_0', '41C301_0', '41C311_0', '41C321_0', '41C371_0', '41C381_0', '41C391_0', '41C401_0', '41C421_0', '41C431_0', '41C441_0', '41C471_0', '41C521_0', '41C551_0', '41C601_0', '41C611_0', '41C621_0', '41C671_0', '41C681_0', '41C711_0', '41C721_0', '41C731_0', '41C741_0', '41P021_1', '41P041_1', '41P051_1', '41P061_1', '41P161_1', '41P211_1', '41P221_1', '41P261_1', '41P451_1', '41P471_1', '41P581_1', '41P641_1', '41P761_1', '42C071_0', '42C092_0', '42C161_0', '42C211_0', '42C221_0', '42C251_0', '42C271_0', '42C291_0', '42C301_0', '42C311_0', '42C321_0', '42C371_0', '42C381_0', '42C391_0', '42C401_0', '42C421_0', '42C431_0', '42C441_0', '42C471_0', '42C521_0', '42C551_0', '42C601_0', '42C611_0', '42C621_0', '42C671_0', '42C681_0', '42C711_0', '42C721_0', '42C731_0', '42C741_0', '42P021_1', '42P041_1', '42P051_1', '42P061_1', '42P161_1', '42P211_1', '42P221_1', '42P261_1', '42P451_1', '42P471_1', '42P581_1', '42P641_1', '42P761_1', '43C071_0', '43C092_0', '43C161_0', '43C211_0', '43C221_0', '43C251_0', '43C271_0', '43C291_0', '43C301_0', '43C311_0', '43C321_0', '43C371_0', '43C381_0', '43C391_0', '43C401_0', '43C421_0', '43C431_0', '43C441_0', '43C471_0', '43C521_0', '43C551_0', '43C601_0', '43C611_0', '43C621_0', '43C671_0', '43C681_0', '43C711_0', '43C721_0', '43C731_0', '43C741_0', '43P021_1', '43P041_1', '43P051_1', '43P061_1', '43P161_1', '43P211_1', '43P221_1', '43P261_1', '43P451_1', '43P471_1', '43P581_1', '43P641_1', '43P761_1', '44C071_0', '44C092_0', '44C161_0', '44C211_0', '44C221_0', '44C251_0', '44C271_0', '44C291_0', '44C301_0', '44C311_0', '44C321_0', '44C371_0', '44C381_0', '44C391_0', '44C401_0', '44C421_0', '44C431_0', '44C441_0', '44C471_0', '44C521_0', '44C551_0', '44C601_0', '44C611_0', '44C621_0', '44C671_0', '44C681_0', '44C711_0', '44C721_0', '44C731_0', '44C741_0', '44P021_1', '44P041_1', '44P051_1', '44P061_1', '44P161_1', '44P211_1', '44P221_1', '44P261_1', '44P451_1', '44P471_1', '44P581_1', '44P641_1', '44P761_1', '45C071_0', '45C092_0', '45C161_0', '45C211_0', '45C221_0', '45C251_0', '45C271_0', '45C301_0', '45C311_0', '45C321_0', '45C371_0', '45C381_0', '45C391_0', '45C401_0', '45C421_0', '45C431_0', '45C441_0', '45C471_0', '45C521_0', '45C551_0', '45C601_0', '45C611_0', '45C621_0', '45C671_0', '45C711_0', '45C721_0', '45C731_0', '45C741_0', '45P021_1', '45P041_1', '45P051_1', '45P061_1', '45P161_1', '45P211_1', '45P221_1', '45P261_1', '45P451_1', '45P471_1', '45P581_1', '45P641_1', '45P761_1', '46C071_0', '46C092_0', '46C161_0', '46C211_0', '46C221_0', '46C251_0', '46C271_0', '46C301_0', '46C311_0', '46C321_0', '46C371_0', '46C381_0', '46C391_0', '46C401_0', '46C421_0', '46C431_0', '46C441_0', '46C471_0', '46C521_0', '46C551_0', '46C601_0', '46C611_0', '46C621_0', '46C671_0', '46C711_0', '46C721_0', '46C731_0', '46C741_0', '46P021_1', '46P041_1', '46P051_1', '46P061_1', '46P161_1', '46P211_1', '46P451_1', '46P471_1', '46P581_1', '46P641_1', '46P761_1', '47C071_0', '47C092_0', '47C161_0', '47C211_0', '47C221_0', '47C251_0', '47C271_0', '47C301_0', '47C311_0', '47C321_0', '47C371_0', '47C381_0', '47C391_0', '47C401_0', '47C421_0', '47C431_0', '47C441_0', '47C471_0', '47C521_0', '47C551_0', '47C601_0', '47C611_0', '47C621_0', '47C671_0', '47C711_0', '47C721_0', '47C731_0', '47C741_0', '47P021_1', '47P041_1', '47P051_1', '47P061_1', '47P161_1', '47P211_1', '47P451_1', '47P471_1', '47P581_1', '47P641_1', '47P761_1', '48C071_0', '48C092_0', '48C161_0', '48C211_0', '48C221_0', '48C251_0', '48C271_0', '48C301_0', '48C311_0', '48C321_0', '48C371_0', '48C381_0', '48C391_0', '48C401_0', '48C421_0', '48C431_0', '48C441_0', '48C471_0', '48C521_0', '48C551_0', '48C601_0', '48C611_0', '48C621_0', '48C671_0', '48C711_0', '48C721_0', '48C731_0', '48C741_0', '48P021_1', '48P051_1', '48P061_1', '48P161_1', '48P211_1', '48P451_1', '48P471_1', '48P581_1', '48P641_1', '48P761_1', '49C071_0', '49C092_0', '49C161_0', '49C211_0', '49C221_0', '49C251_0', '49C271_0', '49C301_0', '49C311_0', '49C321_0', '49C371_0', '49C381_0', '49C391_0', '49C401_0', '49C421_0', '49C431_0', '49C441_0', '49C471_0', '49C521_0', '49C551_0', '49C601_0', '49C611_0', '49C621_0', '49C671_0', '49C711_0', '49C721_0', '49C731_0', '49C741_0', '49P021_1', '49P051_1', '49P061_1', '49P161_1', '49P211_1', '49P451_1', '49P471_1', '49P581_1', '49P641_1', '49P761_1', '4C021_0', '4C042_0', '4C071_0', '4C081_0', '4C092_0', '4C102_0', '4C132_0', '4C151_0', '4C161_0', '4C171_0', '4C181_0', '4C201_0', '4C211_0', '4C221_0', '4C251_0', '4C271_0', '4C281_0', '4C291_0', '4C301_0', '4C311_0', '4C321_0', '4C341_0', '4C361_0', '4C371_0', '4C381_0', '4C391_0', '4C401_0', '4C411_0', '4C421_0', '4C431_0', '4C441_0', '4C471_0', '4C481_0', '4C521_0', '4C551_0', '4C561_0', '4C601_0', '4C611_0', '4C621_0', '4C631_0', '4C641_0', '4C651_0', '4C671_0', '4C681_0', '4C691_0', '4C711_0', '4C721_0', '4C731_0', '4C741_0', '4C771_0', '4P021_1', '4P031_1', '4P041_1', '4P051_1', '4P061_1', '4P091_1', '4P112_2', '4P131_1', '4P142_2', '4P161_1', '4P211_1', '4P221_1', '4P261_1', '4P391_1', '4P411_1', '4P431_1', '4P441_1', '4P451_1', '4P461_1', '4P471_1', '4P501_1', '4P541_1', '4P581_1', '4P641_1', '4P681_1', '4P761_1', '50C071_0', '50C092_0', '50C161_0', '50C211_0', '50C221_0', '50C251_0', '50C271_0', '50C301_0', '50C311_0', '50C321_0', '50C371_0', '50C391_0', '50C401_0', '50C421_0', '50C431_0', '50C441_0', '50C471_0', '50C521_0', '50C551_0', '50C601_0', '50C611_0', '50C621_0', '50C671_0', '50C711_0', '50C721_0', '50C731_0', '50C741_0', '50P021_1', '50P051_1', '50P061_1', '50P161_1', '50P211_1', '50P451_1', '50P471_1', '50P581_1', '50P641_1', '50P761_1', '51C071_0', '51C092_0', '51C161_0', '51C221_0', '51C251_0', '51C271_0', '51C301_0', '51C311_0', '51C321_0', '51C371_0', '51C391_0', '51C401_0', '51C421_0', '51C431_0', '51C441_0', '51C471_0', '51C521_0', '51C551_0', '51C601_0', '51C611_0', '51C621_0', '51C671_0', '51C711_0', '51C721_0', '51C731_0', '51C741_0', '51P021_1', '51P051_1', '51P061_1', '51P161_1', '51P211_1', '51P451_1', '51P471_1', '51P581_1', '51P641_1', '51P761_1', '52C071_0', '52C092_0', '52C161_0', '52C221_0', '52C251_0', '52C271_0', '52C301_0', '52C311_0', '52C321_0', '52C371_0', '52C391_0', '52C401_0', '52C421_0', '52C431_0', '52C441_0', '52C471_0', '52C521_0', '52C551_0', '52C601_0', '52C611_0', '52C621_0', '52C671_0', '52C711_0', '52C721_0', '52C731_0', '52C741_0', '52P021_1', '52P051_1', '52P161_1', '52P211_1', '52P451_1', '52P471_1', '52P581_1', '52P641_1', '52P761_1', '53C071_0', '53C092_0', '53C161_0', '53C221_0', '53C251_0', '53C271_0', '53C301_0', '53C311_0', '53C321_0', '53C371_0', '53C391_0', '53C401_0', '53C421_0', '53C431_0', '53C441_0', '53C471_0', '53C521_0', '53C551_0', '53C601_0', '53C611_0', '53C621_0', '53C671_0', '53C711_0', '53C721_0', '53C731_0', '53C741_0', '53P021_1', '53P051_1', '53P161_1', '53P211_1', '53P451_1', '53P471_1', '53P581_1', '53P641_1', '53P761_1', '54C071_0', '54C161_0', '54C221_0', '54C251_0', '54C271_0', '54C301_0', '54C311_0', '54C321_0', '54C371_0', '54C391_0', '54C401_0', '54C421_0', '54C431_0', '54C441_0', '54C471_0', '54C521_0', '54C551_0', '54C601_0', '54C611_0', '54C621_0', '54C671_0', '54C711_0', '54C721_0', '54C731_0', '54C741_0', '54P021_1', '54P051_1', '54P161_1', '54P211_1', '54P451_1', '54P471_1', '54P581_1', '54P641_1', '54P761_1', '55C071_0', '55C161_0', '55C221_0', '55C271_0', '55C301_0', '55C311_0', '55C321_0', '55C371_0', '55C391_0', '55C401_0', '55C421_0', '55C431_0', '55C441_0', '55C471_0', '55C521_0', '55C551_0', '55C601_0', '55C611_0', '55C621_0', '55C711_0', '55C721_0', '55C731_0', '55C741_0', '55P021_1', '55P051_1', '55P161_1', '55P211_1', '55P451_1', '55P471_1', '55P581_1', '55P641_1', '55P761_1', '56C071_0', '56C161_0', '56C221_0', '56C271_0', '56C301_0', '56C311_0', '56C321_0', '56C371_0', '56C391_0', '56C401_0', '56C421_0', '56C431_0', '56C441_0', '56C471_0', '56C521_0', '56C551_0', '56C601_0', '56C611_0', '56C621_0', '56C711_0', '56C721_0', '56C731_0', '56C741_0', '56P021_1', '56P051_1', '56P161_1', '56P211_1', '56P451_1', '56P471_1', '56P581_1', '56P641_1', '56P761_1', '57C071_0', '57C161_0', '57C221_0', '57C271_0', '57C301_0', '57C311_0', '57C321_0', '57C371_0', '57C391_0', '57C401_0', '57C421_0', '57C431_0', '57C441_0', '57C471_0', '57C521_0', '57C551_0', '57C601_0', '57C611_0', '57C621_0', '57C711_0', '57C721_0', '57C731_0', '57C741_0', '57P021_1', '57P051_1', '57P161_1', '57P211_1', '57P451_1', '57P471_1', '57P581_1', '57P641_1', '57P761_1', '58C071_0', '58C161_0', '58C221_0', '58C271_0', '58C301_0', '58C311_0', '58C321_0', '58C371_0', '58C391_0', '58C401_0', '58C421_0', '58C431_0', '58C441_0', '58C471_0', '58C521_0', '58C611_0', '58C621_0', '58C711_0', '58C721_0', '58C731_0', '58C741_0', '58P021_1', '58P051_1', '58P161_1', '58P211_1', '58P451_1', '58P471_1', '58P581_1', '58P641_1', '58P761_1', '59C071_0', '59C161_0', '59C221_0', '59C271_0', '59C301_0', '59C311_0', '59C321_0', '59C371_0', '59C391_0', '59C401_0', '59C421_0', '59C431_0', '59C441_0', '59C471_0', '59C521_0', '59C611_0', '59C621_0', '59C711_0', '59C721_0', '59C731_0', '59C741_0', '59P021_1', '59P051_1', '59P161_1', '59P211_1', '59P451_1', '59P471_1', '59P581_1', '59P641_1', '59P761_1', '5C021_0', '5C042_0', '5C071_0', '5C081_0', '5C092_0', '5C102_0', '5C132_0', '5C151_0', '5C161_0', '5C171_0', '5C181_0', '5C201_0', '5C211_0', '5C221_0', '5C251_0', '5C271_0', '5C281_0', '5C291_0', '5C301_0', '5C311_0', '5C321_0', '5C341_0', '5C361_0', '5C371_0', '5C381_0', '5C391_0', '5C401_0', '5C411_0', '5C421_0', '5C431_0', '5C441_0', '5C471_0', '5C481_0', '5C521_0', '5C551_0', '5C561_0', '5C601_0', '5C611_0', '5C621_0', '5C631_0', '5C641_0', '5C651_0', '5C671_0', '5C681_0', '5C691_0', '5C711_0', '5C721_0', '5C731_0', '5C741_0', '5C771_0', '5P021_1', '5P031_1', '5P041_1', '5P051_1', '5P061_1', '5P091_1', '5P112_2', '5P131_1', '5P142_2', '5P161_1', '5P211_1', '5P221_1', '5P261_1', '5P391_1', '5P411_1', '5P431_1', '5P441_1', '5P451_1', '5P461_1', '5P471_1', '5P501_1', '5P541_1', '5P581_1', '5P641_1', '5P681_1', '5P761_1', '60C071_0', '60C161_0', '60C221_0', '60C271_0', '60C301_0', '60C311_0', '60C321_0', '60C371_0', '60C391_0', '60C401_0', '60C421_0', '60C431_0', '60C441_0', '60C471_0', '60C521_0', '60C611_0', '60C621_0', '60C711_0', '60C721_0', '60C731_0', '60C741_0', '60P021_1', '60P051_1', '60P161_1', '60P211_1', '60P451_1', '60P471_1', '60P581_1', '60P641_1', '60P761_1', '61C071_0', '61C161_0', '61C221_0', '61C271_0', '61C301_0', '61C311_0', '61C321_0', '61C371_0', '61C391_0', '61C401_0', '61C421_0', '61C431_0', '61C441_0', '61C471_0', '61C521_0', '61C611_0', '61C621_0', '61C711_0', '61C721_0', '61C731_0', '61C741_0', '61P021_1', '61P051_1', '61P161_1', '61P211_1', '61P451_1', '61P471_1', '61P581_1', '61P641_1', '61P761_1', '62C071_0', '62C161_0', '62C221_0', '62C271_0', '62C301_0', '62C311_0', '62C321_0', '62C371_0', '62C391_0', '62C401_0', '62C421_0', '62C431_0', '62C441_0', '62C471_0', '62C521_0', '62C611_0', '62C621_0', '62C711_0', '62C721_0', '62C731_0', '62C741_0', '62P021_1', '62P051_1', '62P161_1', '62P211_1', '62P451_1', '62P471_1', '62P581_1', '62P641_1', '62P761_1', '63C071_0', '63C161_0', '63C221_0', '63C271_0', '63C301_0', '63C311_0', '63C321_0', '63C371_0', '63C391_0', '63C401_0', '63C421_0', '63C431_0', '63C441_0', '63C471_0', '63C521_0', '63C611_0', '63C621_0', '63C711_0', '63C721_0', '63C731_0', '63C741_0', '63P021_1', '63P161_1', '63P211_1', '63P451_1', '63P471_1', '63P581_1', '63P641_1', '63P761_1', '64C071_0', '64C161_0', '64C221_0', '64C271_0', '64C301_0', '64C311_0', '64C321_0', '64C371_0', '64C391_0', '64C401_0', '64C431_0', '64C441_0', '64C471_0', '64C521_0', '64C611_0', '64C621_0', '64C711_0', '64C721_0', '64C731_0', '64C741_0', '64P021_1', '64P161_1', '64P211_1', '64P451_1', '64P471_1', '64P581_1', '64P641_1', '64P761_1', '65C071_0', '65C161_0', '65C221_0', '65C271_0', '65C301_0', '65C311_0', '65C321_0', '65C371_0', '65C391_0', '65C401_0', '65C431_0', '65C441_0', '65C471_0', '65C521_0', '65C611_0', '65C621_0', '65C711_0', '65C721_0', '65C731_0', '65C741_0', '65P021_1', '65P161_1', '65P211_1', '65P451_1', '65P471_1', '65P581_1', '65P641_1', '65P761_1', '66C071_0', '66C161_0', '66C221_0', '66C271_0', '66C301_0', '66C311_0', '66C321_0', '66C371_0', '66C391_0', '66C401_0', '66C431_0', '66C441_0', '66C471_0', '66C521_0', '66C611_0', '66C621_0', '66C711_0', '66C731_0', '66C741_0', '66P021_1', '66P161_1', '66P211_1', '66P451_1', '66P471_1', '66P581_1', '66P641_1', '66P761_1', '67C071_0', '67C161_0', '67C221_0', '67C271_0', '67C301_0', '67C311_0', '67C371_0', '67C391_0', '67C401_0', '67C431_0', '67C441_0', '67C471_0', '67C521_0', '67C611_0', '67C621_0', '67C711_0', '67C731_0', '67C741_0', '67P161_1', '67P211_1', '67P451_1', '67P471_1', '67P581_1', '67P641_1', '67P761_1', '68C071_0', '68C161_0', '68C221_0', '68C271_0', '68C301_0', '68C311_0', '68C371_0', '68C391_0', '68C401_0', '68C431_0', '68C441_0', '68C471_0', '68C521_0', '68C621_0', '68C711_0', '68C731_0', '68C741_0', '68P161_1', '68P211_1', '68P451_1', '68P471_1', '68P581_1', '68P641_1', '68P761_1', '69C071_0', '69C161_0', '69C221_0', '69C271_0', '69C301_0', '69C311_0', '69C371_0', '69C391_0', '69C401_0', '69C431_0', '69C441_0', '69C471_0', '69C521_0', '69C621_0', '69C711_0', '69C731_0', '69C741_0', '69P161_1', '69P211_1', '69P451_1', '69P471_1', '69P581_1', '69P641_1', '69P761_1', '6C021_0', '6C042_0', '6C071_0', '6C081_0', '6C092_0', '6C102_0', '6C132_0', '6C151_0', '6C161_0', '6C171_0', '6C181_0', '6C201_0', '6C211_0', '6C221_0', '6C251_0', '6C271_0', '6C281_0', '6C291_0', '6C301_0', '6C311_0', '6C321_0', '6C341_0', '6C361_0', '6C371_0', '6C381_0', '6C391_0', '6C401_0', '6C411_0', '6C421_0', '6C431_0', '6C441_0', '6C471_0', '6C481_0', '6C521_0', '6C551_0', '6C561_0', '6C601_0', '6C611_0', '6C621_0', '6C631_0', '6C641_0', '6C651_0', '6C671_0', '6C681_0', '6C691_0', '6C711_0', '6C721_0', '6C731_0', '6C741_0', '6C771_0', '6P021_1', '6P031_1', '6P041_1', '6P051_1', '6P061_1', '6P091_1', '6P112_2', '6P131_1', '6P142_2', '6P161_1', '6P211_1', '6P221_1', '6P261_1', '6P391_1', '6P431_1', '6P441_1', '6P451_1', '6P461_1', '6P471_1', '6P501_1', '6P541_1', '6P581_1', '6P641_1', '6P681_1', '6P761_1', '70C071_0', '70C161_0', '70C221_0', '70C271_0', '70C301_0', '70C311_0', '70C371_0', '70C391_0', '70C401_0', '70C431_0', '70C441_0', '70C471_0', '70C521_0', '70C621_0', '70C711_0', '70C731_0', '70C741_0', '70P161_1', '70P211_1', '70P451_1', '70P471_1', '70P581_1', '70P641_1', '70P761_1', '71C071_0', '71C161_0', '71C221_0', '71C271_0', '71C301_0', '71C311_0', '71C371_0', '71C391_0', '71C401_0', '71C431_0', '71C441_0', '71C471_0', '71C521_0', '71C711_0', '71C731_0', '71C741_0', '71P161_1', '71P211_1', '71P451_1', '71P471_1', '71P581_1', '71P641_1', '71P761_1', '72C071_0', '72C161_0', '72C221_0', '72C271_0', '72C301_0', '72C311_0', '72C371_0', '72C391_0', '72C401_0', '72C431_0', '72C441_0', '72C471_0', '72C521_0', '72C711_0', '72C731_0', '72C741_0', '72P161_1', '72P211_1', '72P451_1', '72P471_1', '72P581_1', '72P641_1', '72P761_1', '73C071_0', '73C161_0', '73C221_0', '73C271_0', '73C301_0', '73C311_0', '73C371_0', '73C391_0', '73C401_0', '73C431_0', '73C441_0', '73C471_0', '73C521_0', '73C711_0', '73C731_0', '73P161_1', '73P211_1', '73P451_1', '73P471_1', '73P581_1', '73P641_1', '73P761_1', '74C071_0', '74C161_0', '74C221_0', '74C271_0', '74C301_0', '74C311_0', '74C371_0', '74C391_0', '74C401_0', '74C431_0', '74C441_0', '74C471_0', '74C521_0', '74C711_0', '74C731_0', '74P161_1', '74P211_1', '74P451_1', '74P471_1', '74P581_1', '74P641_1', '74P761_1', '75C071_0', '75C161_0', '75C221_0', '75C271_0', '75C301_0', '75C311_0', '75C371_0', '75C391_0', '75C401_0', '75C431_0', '75C441_0', '75C471_0', '75C521_0', '75C711_0', '75C731_0', '75P161_1', '75P211_1', '75P451_1', '75P471_1', '75P581_1', '75P641_1', '75P761_1', '76C071_0', '76C161_0', '76C221_0', '76C271_0', '76C301_0', '76C311_0', '76C371_0', '76C391_0', '76C401_0', '76C431_0', '76C441_0', '76C471_0', '76C521_0', '76C711_0', '76C731_0', '76P161_1', '76P211_1', '76P451_1', '76P471_1', '76P581_1', '76P641_1', '76P761_1', '77C071_0', '77C161_0', '77C221_0', '77C271_0', '77C301_0', '77C311_0', '77C371_0', '77C391_0', '77C401_0', '77C431_0', '77C441_0', '77C471_0', '77C521_0', '77C711_0', '77C731_0', '77P161_1', '77P211_1', '77P451_1', '77P471_1', '77P581_1', '77P641_1', '77P761_1', '78C071_0', '78C161_0', '78C221_0', '78C271_0', '78C301_0', '78C311_0', '78C371_0', '78C391_0', '78C401_0', '78C431_0', '78C441_0', '78C471_0', '78C521_0', '78C731_0', '78P161_1', '78P211_1', '78P451_1', '78P471_1', '78P581_1', '78P641_1', '78P761_1', '79C071_0', '79C161_0', '79C221_0', '79C271_0', '79C301_0', '79C311_0', '79C371_0', '79C391_0', '79C401_0', '79C431_0', '79C441_0', '79C471_0', '79C521_0', '79C731_0', '79P161_1', '79P211_1', '79P451_1', '79P471_1', '79P581_1', '79P641_1', '79P761_1', '7C021_0', '7C042_0', '7C071_0', '7C081_0', '7C092_0', '7C102_0', '7C132_0', '7C151_0', '7C161_0', '7C171_0', '7C181_0', '7C201_0', '7C211_0', '7C221_0', '7C251_0', '7C271_0', '7C281_0', '7C291_0', '7C301_0', '7C311_0', '7C321_0', '7C341_0', '7C361_0', '7C371_0', '7C381_0', '7C391_0', '7C401_0', '7C411_0', '7C421_0', '7C431_0', '7C441_0', '7C471_0', '7C481_0', '7C521_0', '7C551_0', '7C561_0', '7C601_0', '7C611_0', '7C621_0', '7C631_0', '7C641_0', '7C651_0', '7C671_0', '7C681_0', '7C691_0', '7C711_0', '7C721_0', '7C731_0', '7C741_0', '7C771_0', '7P021_1', '7P031_1', '7P041_1', '7P051_1', '7P061_1', '7P091_1', '7P112_2', '7P131_1', '7P142_2', '7P161_1', '7P211_1', '7P221_1', '7P261_1', '7P391_1', '7P431_1', '7P441_1', '7P451_1', '7P471_1', '7P501_1', '7P541_1', '7P581_1', '7P641_1', '7P681_1', '7P761_1', '80C161_0', '80C221_0', '80C271_0', '80C301_0', '80C311_0', '80C371_0', '80C391_0', '80C401_0', '80C431_0', '80C441_0', '80C471_0', '80C521_0', '80C731_0', '80P161_1', '80P211_1', '80P451_1', '80P471_1', '80P581_1', '80P641_1', '80P761_1', '81C161_0', '81C221_0', '81C271_0', '81C301_0', '81C311_0', '81C371_0', '81C391_0', '81C401_0', '81C431_0', '81C441_0', '81C471_0', '81C731_0', '81P161_1', '81P211_1', '81P451_1', '81P471_1', '81P581_1', '81P641_1', '82C161_0', '82C221_0', '82C271_0', '82C301_0', '82C311_0', '82C371_0', '82C391_0', '82C401_0', '82C431_0', '82C441_0', '82C471_0', '82C731_0', '82P161_1', '82P211_1', '82P451_1', '82P471_1', '82P581_1', '82P641_1', '83C161_0', '83C221_0', '83C271_0', '83C301_0', '83C311_0', '83C371_0', '83C391_0', '83C401_0', '83C431_0', '83C441_0', '83C471_0', '83C731_0', '83P161_1', '83P211_1', '83P451_1', '83P471_1', '83P581_1', '83P641_1', '84C161_0', '84C221_0', '84C271_0', '84C301_0', '84C311_0', '84C371_0', '84C391_0', '84C401_0', '84C431_0', '84C441_0', '84C471_0', '84C731_0', '84P161_1', '84P211_1', '84P451_1', '84P471_1', '84P581_1', '85C161_0', '85C221_0', '85C271_0', '85C301_0', '85C311_0', '85C371_0', '85C391_0', '85C401_0', '85C431_0', '85C441_0', '85C471_0', '85C731_0', '85P161_1', '85P211_1', '85P451_1', '85P471_1', '85P581_1', '86C161_0', '86C221_0', '86C271_0', '86C301_0', '86C311_0', '86C371_0', '86C391_0', '86C401_0', '86C431_0', '86C441_0', '86C471_0', '86P161_1', '86P211_1', '86P451_1', '86P471_1', '86P581_1', '87C161_0', '87C221_0', '87C271_0', '87C301_0', '87C311_0', '87C371_0', '87C391_0', '87C401_0', '87C441_0', '87C471_0', '87P161_1', '87P211_1', '87P451_1', '87P471_1', '87P581_1', '88C161_0', '88C221_0', '88C271_0', '88C301_0', '88C311_0', '88C371_0', '88C391_0', '88C401_0', '88C441_0', '88C471_0', '88P161_1', '88P211_1', '88P451_1', '88P471_1', '88P581_1', '89C161_0', '89C221_0', '89C271_0', '89C301_0', '89C311_0', '89C371_0', '89C391_0', '89C401_0', '89C441_0', '89C471_0', '89P161_1', '89P211_1', '89P451_1', '89P471_1', '89P581_1', '8C021_0', '8C042_0', '8C071_0', '8C081_0', '8C092_0', '8C102_0', '8C151_0', '8C161_0', '8C171_0', '8C181_0', '8C201_0', '8C211_0', '8C221_0', '8C251_0', '8C271_0', '8C281_0', '8C291_0', '8C301_0', '8C311_0', '8C321_0', '8C341_0', '8C371_0', '8C381_0', '8C391_0', '8C401_0', '8C411_0', '8C421_0', '8C431_0', '8C441_0', '8C471_0', '8C481_0', '8C521_0', '8C551_0', '8C561_0', '8C601_0', '8C611_0', '8C621_0', '8C631_0', '8C641_0', '8C651_0', '8C671_0', '8C681_0', '8C691_0', '8C711_0', '8C721_0', '8C731_0', '8C741_0', '8C771_0', '8P021_1', '8P031_1', '8P041_1', '8P051_1', '8P061_1', '8P091_1', '8P112_2', '8P131_1', '8P142_2', '8P161_1', '8P211_1', '8P221_1', '8P261_1', '8P391_1', '8P431_1', '8P441_1', '8P451_1', '8P471_1', '8P501_1', '8P541_1', '8P581_1', '8P641_1', '8P681_1', '8P761_1', '90C161_0', '90C221_0', '90C271_0', '90C301_0', '90C311_0', '90C371_0', '90C391_0', '90C401_0', '90C441_0', '90C471_0', '90P161_1', '90P211_1', '90P451_1', '90P471_1', '90P581_1', '91C161_0', '91C221_0', '91C271_0', '91C301_0', '91C311_0', '91C371_0', '91C391_0', '91C401_0', '91C441_0', '91C471_0', '91P161_1', '91P211_1', '91P451_1', '91P471_1', '91P581_1', '92C161_0', '92C221_0', '92C271_0', '92C301_0', '92C311_0', '92C371_0', '92C391_0', '92C401_0', '92C441_0', '92C471_0', '92P161_1', '92P211_1', '92P451_1', '92P471_1', '92P581_1', '93C161_0', '93C221_0', '93C271_0', '93C301_0', '93C311_0', '93C371_0', '93C391_0', '93C401_0', '93C441_0', '93C471_0', '93P161_1', '93P211_1', '93P451_1', '93P471_1', '93P581_1', '94C161_0', '94C221_0', '94C271_0', '94C301_0', '94C311_0', '94C371_0', '94C391_0', '94C401_0', '94C441_0', '94C471_0', '94P161_1', '94P211_1', '94P451_1', '94P471_1', '94P581_1', '95C161_0', '95C221_0', '95C271_0', '95C301_0', '95C311_0', '95C371_0', '95C391_0', '95C401_0', '95C441_0', '95C471_0', '95P161_1', '95P211_1', '95P451_1', '95P471_1', '95P581_1', '96C161_0', '96C221_0', '96C271_0', '96C301_0', '96C311_0', '96C371_0', '96C391_0', '96C401_0', '96C441_0', '96C471_0', '96P161_1', '96P211_1', '96P451_1', '96P471_1', '96P581_1', '97C161_0', '97C221_0', '97C271_0', '97C301_0', '97C311_0', '97C371_0', '97C391_0', '97C401_0', '97C441_0', '97C471_0', '97P161_1', '97P211_1', '97P451_1', '97P471_1', '97P581_1', '98C161_0', '98C221_0', '98C271_0', '98C301_0', '98C311_0', '98C371_0', '98C391_0', '98C401_0', '98C441_0', '98C471_0', '98P161_1', '98P211_1', '98P451_1', '98P471_1', '98P581_1', '99C161_0', '99C221_0', '99C271_0', '99C301_0', '99C311_0', '99C371_0', '99C391_0', '99C401_0', '99C441_0', '99C471_0', '99P161_1', '99P211_1', '99P451_1', '99P471_1', '99P581_1', '9C021_0', '9C042_0', '9C071_0', '9C081_0', '9C092_0', '9C102_0', '9C151_0', '9C161_0', '9C171_0', '9C181_0', '9C201_0', '9C211_0', '9C221_0', '9C251_0', '9C271_0', '9C281_0', '9C291_0', '9C301_0', '9C311_0', '9C321_0', '9C341_0', '9C371_0', '9C381_0', '9C391_0', '9C401_0', '9C421_0', '9C431_0', '9C441_0', '9C471_0', '9C481_0', '9C521_0', '9C551_0', '9C561_0', '9C601_0', '9C611_0', '9C621_0', '9C631_0', '9C641_0', '9C671_0', '9C681_0', '9C691_0', '9C711_0', '9C721_0', '9C731_0', '9C741_0', '9C771_0', '9P021_1', '9P031_1', '9P041_1', '9P051_1', '9P061_1', '9P091_1', '9P112_2', '9P131_1', '9P142_2', '9P161_1', '9P211_1', '9P221_1', '9P261_1', '9P391_1', '9P431_1', '9P441_1', '9P451_1', '9P471_1', '9P501_1', '9P541_1', '9P581_1', '9P641_1', '9P681_1', '9P761_1']
filenames2 = ['0C022_0', '0C041_0', '0C072_0', '0C082_0', '0C091_0', '0C101_0', '0C131_0', '0C152_0', '0C162_0', '0C172_0', '0C182_0', '0C202_0', '0C212_0', '0C222_0', '0C272_0', '0C282_0', '0C292_0', '0C302_0', '0C312_0', '0C322_0', '0C342_0', '0C372_0', '0C382_0', '0C392_0', '0C402_0', '0C412_0', '0C422_0', '0C432_0', '0C442_0', '0C472_0', '0C482_0', '0C502_0', '0C522_0', '0C562_0', '0C582_0', '0C602_0', '0C612_0', '0C622_0', '0C632_0', '0C652_0', '0C672_0', '0C682_0', '0C712_0', '0C732_0', '0C742_0', '0P042_2', '0P052_2', '0P062_2', '0P071_1', '0P092_2', '0P111_1', '0P132_2', '0P141_1', '0P162_2', '0P201_1', '0P212_2', '0P332_2', '0P352_2', '0P412_2', '0P422_2', '0P432_2', '0P452_2', '0P542_2', '0P642_2', '0P672_2', '0P692_2', '0P712_2', '0P992_2', '100C091_0', '100C152_0', '100C162_0', '100C272_0', '100C302_0', '100C312_0', '100C392_0', '100C402_0', '100C502_0', '101C091_0', '101C152_0', '101C162_0', '101C272_0', '101C302_0', '101C312_0', '101C392_0', '101C402_0', '101C502_0', '102C091_0', '102C152_0', '102C162_0', '102C272_0', '102C302_0', '102C312_0', '102C392_0', '102C402_0', '102C502_0', '103C152_0', '103C162_0', '103C272_0', '103C302_0', '103C312_0', '103C392_0', '103C402_0', '103C502_0', '104C152_0', '104C162_0', '104C272_0', '104C302_0', '104C312_0', '104C392_0', '104C402_0', '104C502_0', '105C152_0', '105C162_0', '105C272_0', '105C302_0', '105C312_0', '105C392_0', '105C402_0', '105C502_0', '106C152_0', '106C162_0', '106C272_0', '106C302_0', '106C312_0', '106C392_0', '106C402_0', '106C502_0', '107C152_0', '107C162_0', '107C272_0', '107C302_0', '107C312_0', '107C392_0', '107C402_0', '107C502_0', '108C152_0', '108C162_0', '108C272_0', '108C302_0', '108C312_0', '108C392_0', '108C402_0', '108C502_0', '109C152_0', '109C162_0', '109C272_0', '109C302_0', '109C312_0', '109C392_0', '109C402_0', '109C502_0', '10C041_0', '10C072_0', '10C082_0', '10C091_0', '10C101_0', '10C131_0', '10C152_0', '10C162_0', '10C172_0', '10C182_0', '10C202_0', '10C212_0', '10C222_0', '10C272_0', '10C282_0', '10C292_0', '10C302_0', '10C312_0', '10C322_0', '10C342_0', '10C372_0', '10C382_0', '10C392_0', '10C402_0', '10C412_0', '10C422_0', '10C432_0', '10C442_0', '10C502_0', '10C522_0', '10C562_0', '10C582_0', '10C602_0', '10C612_0', '10C622_0', '10C652_0', '10C672_0', '10C682_0', '10C712_0', '10C732_0', '10C742_0', '10P052_2', '10P071_1', '10P111_1', '10P132_2', '10P162_2', '10P201_1', '10P212_2', '10P332_2', '10P412_2', '10P422_2', '10P542_2', '10P642_2', '10P672_2', '10P712_2', '110C152_0', '110C272_0', '110C302_0', '110C312_0', '110C392_0', '110C402_0', '110C502_0', '111C152_0', '111C272_0', '111C302_0', '111C312_0', '111C392_0', '111C402_0', '111C502_0', '112C152_0', '112C272_0', '112C302_0', '112C312_0', '112C392_0', '112C402_0', '112C502_0', '113C152_0', '113C272_0', '113C302_0', '113C312_0', '113C392_0', '113C402_0', '113C502_0', '114C152_0', '114C272_0', '114C302_0', '114C312_0', '114C392_0', '114C402_0', '114C502_0', '115C152_0', '115C272_0', '115C302_0', '115C312_0', '115C392_0', '115C402_0', '115C502_0', '116C152_0', '116C302_0', '116C312_0', '116C392_0', '116C402_0', '116C502_0', '117C152_0', '117C302_0', '117C312_0', '117C392_0', '117C402_0', '117C502_0', '118C302_0', '118C312_0', '118C392_0', '118C402_0', '118C502_0', '119C302_0', '119C312_0', '119C392_0', '119C402_0', '119C502_0', '11C041_0', '11C072_0', '11C082_0', '11C091_0', '11C101_0', '11C131_0', '11C152_0', '11C162_0', '11C172_0', '11C182_0', '11C202_0', '11C212_0', '11C222_0', '11C272_0', '11C282_0', '11C292_0', '11C302_0', '11C312_0', '11C322_0', '11C342_0', '11C372_0', '11C382_0', '11C392_0', '11C402_0', '11C412_0', '11C422_0', '11C432_0', '11C442_0', '11C502_0', '11C522_0', '11C562_0', '11C582_0', '11C602_0', '11C612_0', '11C622_0', '11C652_0', '11C672_0', '11C682_0', '11C712_0', '11C732_0', '11C742_0', '11P052_2', '11P071_1', '11P111_1', '11P132_2', '11P162_2', '11P201_1', '11P212_2', '11P332_2', '11P412_2', '11P422_2', '11P542_2', '11P642_2', '11P672_2', '11P712_2', '120C302_0', '120C312_0', '120C392_0', '120C402_0', '120C502_0', '121C302_0', '121C312_0', '121C392_0', '121C402_0', '121C502_0', '122C302_0', '122C312_0', '122C392_0', '122C402_0', '122C502_0', '123C302_0', '123C312_0', '123C392_0', '123C402_0', '123C502_0', '124C302_0', '124C312_0', '124C392_0', '124C402_0', '124C502_0', '125C302_0', '125C312_0', '125C392_0', '125C402_0', '125C502_0', '126C302_0', '126C312_0', '126C392_0', '126C402_0', '126C502_0', '127C302_0', '127C312_0', '127C392_0', '127C402_0', '127C502_0', '128C302_0', '128C312_0', '128C392_0', '128C402_0', '128C502_0', '129C302_0', '129C312_0', '129C392_0', '129C402_0', '129C502_0', '12C041_0', '12C072_0', '12C082_0', '12C091_0', '12C101_0', '12C131_0', '12C152_0', '12C162_0', '12C172_0', '12C182_0', '12C212_0', '12C222_0', '12C272_0', '12C282_0', '12C292_0', '12C302_0', '12C312_0', '12C322_0', '12C342_0', '12C372_0', '12C382_0', '12C392_0', '12C402_0', '12C412_0', '12C422_0', '12C432_0', '12C442_0', '12C502_0', '12C522_0', '12C562_0', '12C582_0', '12C602_0', '12C612_0', '12C622_0', '12C652_0', '12C672_0', '12C682_0', '12C712_0', '12C732_0', '12C742_0', '12P052_2', '12P071_1', '12P111_1', '12P132_2', '12P162_2', '12P201_1', '12P212_2', '12P332_2', '12P412_2', '12P422_2', '12P542_2', '12P642_2', '12P672_2', '12P712_2', '130C302_0', '130C312_0', '130C392_0', '130C402_0', '130C502_0', '131C302_0', '131C312_0', '131C402_0', '131C502_0', '132C302_0', '132C312_0', '132C402_0', '132C502_0', '133C302_0', '133C312_0', '133C402_0', '133C502_0', '134C302_0', '134C312_0', '134C402_0', '134C502_0', '135C302_0', '135C312_0', '135C402_0', '135C502_0', '136C302_0', '136C312_0', '136C402_0', '136C502_0', '137C302_0', '137C312_0', '137C402_0', '137C502_0', '138C302_0', '138C312_0', '138C402_0', '138C502_0', '139C302_0', '139C312_0', '139C402_0', '139C502_0', '13C041_0', '13C072_0', '13C082_0', '13C091_0', '13C101_0', '13C131_0', '13C152_0', '13C162_0', '13C172_0', '13C182_0', '13C212_0', '13C222_0', '13C272_0', '13C282_0', '13C292_0', '13C302_0', '13C312_0', '13C322_0', '13C342_0', '13C372_0', '13C382_0', '13C392_0', '13C402_0', '13C412_0', '13C422_0', '13C432_0', '13C442_0', '13C502_0', '13C522_0', '13C562_0', '13C582_0', '13C602_0', '13C612_0', '13C622_0', '13C652_0', '13C672_0', '13C682_0', '13C712_0', '13C732_0', '13C742_0', '13P052_2', '13P071_1', '13P111_1', '13P132_2', '13P162_2', '13P201_1', '13P212_2', '13P412_2', '13P422_2', '13P542_2', '13P642_2', '13P672_2', '13P712_2', '140C302_0', '140C312_0', '140C402_0', '140C502_0', '141C302_0', '141C312_0', '141C402_0', '141C502_0', '142C302_0', '142C312_0', '142C402_0', '142C502_0', '143C302_0', '143C312_0', '143C402_0', '143C502_0', '144C302_0', '144C312_0', '144C402_0', '144C502_0', '145C302_0', '145C312_0', '145C402_0', '145C502_0', '146C302_0', '146C312_0', '146C402_0', '146C502_0', '147C302_0', '147C312_0', '147C402_0', '147C502_0', '148C302_0', '148C312_0', '148C502_0', '149C302_0', '149C312_0', '149C502_0', '14C041_0', '14C072_0', '14C082_0', '14C091_0', '14C101_0', '14C131_0', '14C152_0', '14C162_0', '14C172_0', '14C182_0', '14C212_0', '14C222_0', '14C272_0', '14C282_0', '14C292_0', '14C302_0', '14C312_0', '14C322_0', '14C342_0', '14C372_0', '14C382_0', '14C392_0', '14C402_0', '14C412_0', '14C422_0', '14C432_0', '14C442_0', '14C502_0', '14C522_0', '14C562_0', '14C582_0', '14C602_0', '14C612_0', '14C622_0', '14C652_0', '14C672_0', '14C682_0', '14C712_0', '14C732_0', '14C742_0', '14P052_2', '14P071_1', '14P111_1', '14P132_2', '14P162_2', '14P201_1', '14P212_2', '14P412_2', '14P422_2', '14P542_2', '14P642_2', '14P672_2', '14P712_2', '150C302_0', '150C312_0', '150C502_0', '151C302_0', '151C312_0', '151C502_0', '152C302_0', '152C312_0', '152C502_0', '153C302_0', '153C312_0', '153C502_0', '154C302_0', '154C312_0', '154C502_0', '155C302_0', '155C312_0', '155C502_0', '156C302_0', '156C312_0', '156C502_0', '157C302_0', '157C312_0', '157C502_0', '158C302_0', '158C312_0', '158C502_0', '159C302_0', '159C312_0', '159C502_0', '15C041_0', '15C072_0', '15C082_0', '15C091_0', '15C131_0', '15C152_0', '15C162_0', '15C172_0', '15C182_0', '15C212_0', '15C222_0', '15C272_0', '15C282_0', '15C292_0', '15C302_0', '15C312_0', '15C322_0', '15C342_0', '15C372_0', '15C382_0', '15C392_0', '15C402_0', '15C412_0', '15C422_0', '15C432_0', '15C442_0', '15C502_0', '15C522_0', '15C562_0', '15C582_0', '15C602_0', '15C612_0', '15C622_0', '15C652_0', '15C672_0', '15C682_0', '15C712_0', '15C732_0', '15C742_0', '15P052_2', '15P071_1', '15P111_1', '15P132_2', '15P162_2', '15P201_1', '15P212_2', '15P412_2', '15P422_2', '15P542_2', '15P642_2', '15P672_2', '15P712_2', '160C302_0', '160C312_0', '161C302_0', '161C312_0', '162C302_0', '162C312_0', '163C302_0', '163C312_0', '164C302_0', '164C312_0', '165C302_0', '165C312_0', '166C302_0', '166C312_0', '167C302_0', '167C312_0', '168C302_0', '168C312_0', '169C302_0', '169C312_0', '16C041_0', '16C072_0', '16C082_0', '16C091_0', '16C131_0', '16C152_0', '16C162_0', '16C172_0', '16C182_0', '16C212_0', '16C222_0', '16C272_0', '16C282_0', '16C292_0', '16C302_0', '16C312_0', '16C322_0', '16C342_0', '16C372_0', '16C382_0', '16C392_0', '16C402_0', '16C412_0', '16C422_0', '16C432_0', '16C442_0', '16C502_0', '16C522_0', '16C562_0', '16C582_0', '16C602_0', '16C622_0', '16C652_0', '16C672_0', '16C682_0', '16C712_0', '16C732_0', '16C742_0', '16P052_2', '16P071_1', '16P111_1', '16P132_2', '16P162_2', '16P212_2', '16P412_2', '16P422_2', '16P542_2', '16P642_2', '16P712_2', '170C302_0', '170C312_0', '171C302_0', '171C312_0', '172C302_0', '172C312_0', '173C302_0', '173C312_0', '174C302_0', '174C312_0', '175C302_0', '175C312_0', '176C302_0', '176C312_0', '177C302_0', '177C312_0', '178C312_0', '179C312_0', '17C041_0', '17C072_0', '17C082_0', '17C091_0', '17C131_0', '17C152_0', '17C162_0', '17C172_0', '17C212_0', '17C222_0', '17C272_0', '17C282_0', '17C292_0', '17C302_0', '17C312_0', '17C322_0', '17C342_0', '17C372_0', '17C382_0', '17C392_0', '17C402_0', '17C412_0', '17C422_0', '17C432_0', '17C442_0', '17C502_0', '17C522_0', '17C562_0', '17C582_0', '17C602_0', '17C622_0', '17C652_0', '17C672_0', '17C682_0', '17C712_0', '17C732_0', '17C742_0', '17P052_2', '17P071_1', '17P111_1', '17P132_2', '17P162_2', '17P212_2', '17P412_2', '17P542_2', '17P642_2', '17P712_2', '180C312_0', '181C312_0', '182C312_0', '183C312_0', '184C312_0', '185C312_0', '186C312_0', '187C312_0', '188C312_0', '189C312_0', '18C041_0', '18C072_0', '18C082_0', '18C091_0', '18C131_0', '18C152_0', '18C162_0', '18C172_0', '18C212_0', '18C222_0', '18C272_0', '18C282_0', '18C292_0', '18C302_0', '18C312_0', '18C322_0', '18C342_0', '18C372_0', '18C382_0', '18C392_0', '18C402_0', '18C412_0', '18C422_0', '18C432_0', '18C442_0', '18C502_0', '18C522_0', '18C562_0', '18C582_0', '18C602_0', '18C622_0', '18C652_0', '18C672_0', '18C682_0', '18C712_0', '18C732_0', '18C742_0', '18P052_2', '18P071_1', '18P111_1', '18P132_2', '18P162_2', '18P212_2', '18P412_2', '18P542_2', '18P642_2', '18P712_2', '190C312_0', '191C312_0', '192C312_0', '193C312_0', '194C312_0', '195C312_0', '196C312_0', '197C312_0', '198C312_0', '199C312_0', '19C041_0', '19C072_0', '19C082_0', '19C091_0', '19C131_0', '19C152_0', '19C162_0', '19C172_0', '19C212_0', '19C222_0', '19C272_0', '19C282_0', '19C302_0', '19C312_0', '19C322_0', '19C342_0', '19C372_0', '19C382_0', '19C392_0', '19C402_0', '19C412_0', '19C422_0', '19C432_0', '19C442_0', '19C502_0', '19C522_0', '19C562_0', '19C582_0', '19C602_0', '19C622_0', '19C652_0', '19C672_0', '19C682_0', '19C712_0', '19C732_0', '19C742_0', '19P052_2', '19P071_1', '19P111_1', '19P132_2', '19P162_2', '19P212_2', '19P412_2', '19P542_2', '19P642_2', '19P712_2', '1C022_0', '1C041_0', '1C072_0', '1C082_0', '1C091_0', '1C101_0', '1C131_0', '1C152_0', '1C162_0', '1C172_0', '1C182_0', '1C202_0', '1C212_0', '1C222_0', '1C272_0', '1C282_0', '1C292_0', '1C302_0', '1C312_0', '1C322_0', '1C342_0', '1C372_0', '1C382_0', '1C392_0', '1C402_0', '1C412_0', '1C422_0', '1C432_0', '1C442_0', '1C472_0', '1C482_0', '1C502_0', '1C522_0', '1C562_0', '1C582_0', '1C602_0', '1C612_0', '1C622_0', '1C632_0', '1C652_0', '1C672_0', '1C682_0', '1C712_0', '1C732_0', '1C742_0', '1P042_2', '1P052_2', '1P062_2', '1P071_1', '1P092_2', '1P111_1', '1P132_2', '1P141_1', '1P162_2', '1P201_1', '1P212_2', '1P332_2', '1P352_2', '1P412_2', '1P422_2', '1P432_2', '1P452_2', '1P542_2', '1P642_2', '1P672_2', '1P692_2', '1P712_2', '1P992_2', '200C312_0', '201C312_0', '202C312_0', '203C312_0', '204C312_0', '205C312_0', '206C312_0', '207C312_0', '208C312_0', '209C312_0', '20C041_0', '20C072_0', '20C082_0', '20C091_0', '20C131_0', '20C152_0', '20C162_0', '20C172_0', '20C212_0', '20C222_0', '20C272_0', '20C282_0', '20C302_0', '20C312_0', '20C322_0', '20C342_0', '20C372_0', '20C382_0', '20C392_0', '20C402_0', '20C412_0', '20C422_0', '20C432_0', '20C442_0', '20C502_0', '20C522_0', '20C562_0', '20C582_0', '20C602_0', '20C622_0', '20C652_0', '20C672_0', '20C682_0', '20C712_0', '20C732_0', '20C742_0', '20P052_2', '20P071_1', '20P111_1', '20P132_2', '20P162_2', '20P212_2', '20P412_2', '20P542_2', '20P642_2', '20P712_2', '210C312_0', '211C312_0', '212C312_0', '213C312_0', '214C312_0', '215C312_0', '216C312_0', '217C312_0', '218C312_0', '219C312_0', '21C041_0', '21C072_0', '21C082_0', '21C091_0', '21C131_0', '21C152_0', '21C162_0', '21C172_0', '21C212_0', '21C222_0', '21C272_0', '21C282_0', '21C302_0', '21C312_0', '21C322_0', '21C342_0', '21C372_0', '21C382_0', '21C392_0', '21C402_0', '21C412_0', '21C422_0', '21C432_0', '21C442_0', '21C502_0', '21C522_0', '21C562_0', '21C582_0', '21C602_0', '21C622_0', '21C652_0', '21C672_0', '21C682_0', '21C712_0', '21C732_0', '21C742_0', '21P052_2', '21P071_1', '21P111_1', '21P132_2', '21P162_2', '21P212_2', '21P412_2', '21P542_2', '21P642_2', '21P712_2', '220C312_0', '221C312_0', '22C041_0', '22C072_0', '22C082_0', '22C091_0', '22C131_0', '22C152_0', '22C162_0', '22C172_0', '22C212_0', '22C222_0', '22C272_0', '22C282_0', '22C302_0', '22C312_0', '22C342_0', '22C372_0', '22C382_0', '22C392_0', '22C402_0', '22C412_0', '22C422_0', '22C432_0', '22C442_0', '22C502_0', '22C522_0', '22C562_0', '22C582_0', '22C602_0', '22C622_0', '22C652_0', '22C672_0', '22C682_0', '22C712_0', '22C732_0', '22C742_0', '22P052_2', '22P071_1', '22P111_1', '22P132_2', '22P162_2', '22P212_2', '22P412_2', '22P542_2', '22P642_2', '22P712_2', '23C041_0', '23C072_0', '23C082_0', '23C091_0', '23C131_0', '23C152_0', '23C162_0', '23C172_0', '23C212_0', '23C222_0', '23C272_0', '23C282_0', '23C302_0', '23C312_0', '23C342_0', '23C372_0', '23C382_0', '23C392_0', '23C402_0', '23C412_0', '23C422_0', '23C432_0', '23C442_0', '23C502_0', '23C522_0', '23C562_0', '23C582_0', '23C602_0', '23C622_0', '23C652_0', '23C672_0', '23C682_0', '23C712_0', '23C732_0', '23C742_0', '23P052_2', '23P071_1', '23P111_1', '23P132_2', '23P162_2', '23P212_2', '23P412_2', '23P542_2', '23P642_2', '23P712_2', '24C041_0', '24C072_0', '24C082_0', '24C091_0', '24C131_0', '24C152_0', '24C162_0', '24C172_0', '24C212_0', '24C222_0', '24C272_0', '24C282_0', '24C302_0', '24C312_0', '24C342_0', '24C372_0', '24C382_0', '24C392_0', '24C402_0', '24C412_0', '24C422_0', '24C432_0', '24C442_0', '24C502_0', '24C522_0', '24C562_0', '24C582_0', '24C602_0', '24C622_0', '24C652_0', '24C672_0', '24C682_0', '24C712_0', '24C732_0', '24C742_0', '24P052_2', '24P071_1', '24P132_2', '24P162_2', '24P212_2', '24P412_2', '24P542_2', '24P642_2', '24P712_2', '25C041_0', '25C072_0', '25C082_0', '25C091_0', '25C131_0', '25C152_0', '25C162_0', '25C172_0', '25C212_0', '25C222_0', '25C272_0', '25C282_0', '25C302_0', '25C312_0', '25C342_0', '25C372_0', '25C382_0', '25C392_0', '25C402_0', '25C412_0', '25C422_0', '25C432_0', '25C442_0', '25C502_0', '25C522_0', '25C562_0', '25C582_0', '25C602_0', '25C622_0', '25C652_0', '25C672_0', '25C682_0', '25C712_0', '25C732_0', '25C742_0', '25P052_2', '25P071_1', '25P132_2', '25P162_2', '25P212_2', '25P412_2', '25P542_2', '25P642_2', '25P712_2', '26C041_0', '26C072_0', '26C082_0', '26C091_0', '26C131_0', '26C152_0', '26C162_0', '26C172_0', '26C222_0', '26C272_0', '26C282_0', '26C302_0', '26C312_0', '26C342_0', '26C372_0', '26C382_0', '26C392_0', '26C402_0', '26C412_0', '26C422_0', '26C432_0', '26C442_0', '26C502_0', '26C522_0', '26C562_0', '26C602_0', '26C622_0', '26C652_0', '26C672_0', '26C682_0', '26C712_0', '26C732_0', '26C742_0', '26P071_1', '26P132_2', '26P162_2', '26P212_2', '26P412_2', '26P542_2', '26P642_2', '27C041_0', '27C072_0', '27C082_0', '27C091_0', '27C131_0', '27C152_0', '27C162_0', '27C172_0', '27C222_0', '27C272_0', '27C282_0', '27C302_0', '27C312_0', '27C342_0', '27C372_0', '27C382_0', '27C392_0', '27C402_0', '27C412_0', '27C422_0', '27C432_0', '27C442_0', '27C502_0', '27C522_0', '27C562_0', '27C602_0', '27C622_0', '27C672_0', '27C712_0', '27C732_0', '27C742_0', '27P071_1', '27P132_2', '27P162_2', '27P212_2', '27P412_2', '27P542_2', '27P642_2', '28C041_0', '28C072_0', '28C082_0', '28C091_0', '28C131_0', '28C152_0', '28C162_0', '28C172_0', '28C222_0', '28C272_0', '28C282_0', '28C302_0', '28C312_0', '28C342_0', '28C372_0', '28C382_0', '28C392_0', '28C402_0', '28C412_0', '28C422_0', '28C432_0', '28C442_0', '28C502_0', '28C522_0', '28C562_0', '28C602_0', '28C622_0', '28C672_0', '28C712_0', '28C732_0', '28C742_0', '28P071_1', '28P132_2', '28P162_2', '28P212_2', '28P412_2', '28P542_2', '28P642_2', '29C041_0', '29C072_0', '29C082_0', '29C091_0', '29C131_0', '29C152_0', '29C162_0', '29C222_0', '29C272_0', '29C282_0', '29C302_0', '29C312_0', '29C342_0', '29C372_0', '29C382_0', '29C392_0', '29C402_0', '29C412_0', '29C422_0', '29C432_0', '29C442_0', '29C502_0', '29C522_0', '29C562_0', '29C602_0', '29C622_0', '29C672_0', '29C712_0', '29C732_0', '29C742_0', '29P071_1', '29P132_2', '29P162_2', '29P212_2', '29P412_2', '29P542_2', '29P642_2', '2C022_0', '2C041_0', '2C072_0', '2C082_0', '2C091_0', '2C101_0', '2C131_0', '2C152_0', '2C162_0', '2C172_0', '2C182_0', '2C202_0', '2C212_0', '2C222_0', '2C272_0', '2C282_0', '2C292_0', '2C302_0', '2C312_0', '2C322_0', '2C342_0', '2C372_0', '2C382_0', '2C392_0', '2C402_0', '2C412_0', '2C422_0', '2C432_0', '2C442_0', '2C472_0', '2C502_0', '2C522_0', '2C562_0', '2C582_0', '2C602_0', '2C612_0', '2C622_0', '2C632_0', '2C652_0', '2C672_0', '2C682_0', '2C712_0', '2C732_0', '2C742_0', '2P042_2', '2P052_2', '2P062_2', '2P071_1', '2P092_2', '2P111_1', '2P132_2', '2P141_1', '2P162_2', '2P201_1', '2P212_2', '2P332_2', '2P352_2', '2P412_2', '2P422_2', '2P432_2', '2P452_2', '2P542_2', '2P642_2', '2P672_2', '2P692_2', '2P712_2', '30C072_0', '30C082_0', '30C091_0', '30C131_0', '30C152_0', '30C162_0', '30C222_0', '30C272_0', '30C282_0', '30C302_0', '30C312_0', '30C342_0', '30C372_0', '30C382_0', '30C392_0', '30C402_0', '30C412_0', '30C422_0', '30C432_0', '30C442_0', '30C502_0', '30C522_0', '30C562_0', '30C602_0', '30C622_0', '30C672_0', '30C712_0', '30C732_0', '30C742_0', '30P071_1', '30P132_2', '30P162_2', '30P212_2', '30P412_2', '30P542_2', '30P642_2', '31C072_0', '31C082_0', '31C091_0', '31C131_0', '31C152_0', '31C162_0', '31C222_0', '31C272_0', '31C282_0', '31C302_0', '31C312_0', '31C342_0', '31C372_0', '31C382_0', '31C392_0', '31C402_0', '31C412_0', '31C422_0', '31C432_0', '31C442_0', '31C502_0', '31C522_0', '31C562_0', '31C602_0', '31C622_0', '31C672_0', '31C712_0', '31C732_0', '31C742_0', '31P071_1', '31P132_2', '31P162_2', '31P212_2', '31P412_2', '31P542_2', '31P642_2', '32C072_0', '32C082_0', '32C091_0', '32C131_0', '32C152_0', '32C162_0', '32C272_0', '32C282_0', '32C302_0', '32C312_0', '32C342_0', '32C372_0', '32C382_0', '32C392_0', '32C402_0', '32C412_0', '32C422_0', '32C442_0', '32C502_0', '32C522_0', '32C562_0', '32C602_0', '32C622_0', '32C672_0', '32C712_0', '32C732_0', '32C742_0', '32P071_1', '32P132_2', '32P162_2', '32P212_2', '32P412_2', '32P542_2', '32P642_2', '33C072_0', '33C082_0', '33C091_0', '33C131_0', '33C152_0', '33C162_0', '33C272_0', '33C282_0', '33C302_0', '33C312_0', '33C342_0', '33C372_0', '33C382_0', '33C392_0', '33C402_0', '33C412_0', '33C422_0', '33C442_0', '33C502_0', '33C522_0', '33C562_0', '33C602_0', '33C622_0', '33C712_0', '33C732_0', '33C742_0', '33P071_1', '33P132_2', '33P162_2', '33P212_2', '33P412_2', '33P542_2', '33P642_2', '34C072_0', '34C082_0', '34C091_0', '34C131_0', '34C152_0', '34C162_0', '34C272_0', '34C282_0', '34C302_0', '34C312_0', '34C342_0', '34C372_0', '34C382_0', '34C392_0', '34C402_0', '34C412_0', '34C422_0', '34C442_0', '34C502_0', '34C522_0', '34C562_0', '34C602_0', '34C622_0', '34C712_0', '34C732_0', '34C742_0', '34P071_1', '34P132_2', '34P162_2', '34P212_2', '34P412_2', '34P542_2', '34P642_2', '35C072_0', '35C082_0', '35C091_0', '35C131_0', '35C152_0', '35C162_0', '35C272_0', '35C282_0', '35C302_0', '35C312_0', '35C342_0', '35C372_0', '35C382_0', '35C392_0', '35C402_0', '35C412_0', '35C422_0', '35C442_0', '35C502_0', '35C522_0', '35C562_0', '35C602_0', '35C622_0', '35C712_0', '35C732_0', '35C742_0', '35P071_1', '35P132_2', '35P212_2', '35P412_2', '35P542_2', '35P642_2', '36C072_0', '36C082_0', '36C091_0', '36C131_0', '36C152_0', '36C162_0', '36C272_0', '36C282_0', '36C302_0', '36C312_0', '36C342_0', '36C372_0', '36C392_0', '36C402_0', '36C412_0', '36C422_0', '36C442_0', '36C502_0', '36C522_0', '36C562_0', '36C602_0', '36C622_0', '36C712_0', '36C732_0', '36C742_0', '36P071_1', '36P132_2', '36P212_2', '36P412_2', '36P542_2', '36P642_2', '37C072_0', '37C082_0', '37C091_0', '37C131_0', '37C152_0', '37C162_0', '37C272_0', '37C282_0', '37C302_0', '37C312_0', '37C342_0', '37C372_0', '37C392_0', '37C402_0', '37C412_0', '37C422_0', '37C442_0', '37C502_0', '37C522_0', '37C602_0', '37C622_0', '37C712_0', '37C732_0', '37C742_0', '37P071_1', '37P132_2', '37P212_2', '37P412_2', '37P542_2', '37P642_2', '38C072_0', '38C082_0', '38C091_0', '38C131_0', '38C152_0', '38C162_0', '38C272_0', '38C282_0', '38C302_0', '38C312_0', '38C342_0', '38C372_0', '38C392_0', '38C402_0', '38C412_0', '38C422_0', '38C442_0', '38C502_0', '38C522_0', '38C622_0', '38C712_0', '38C732_0', '38C742_0', '38P071_1', '38P132_2', '38P212_2', '38P412_2', '38P542_2', '38P642_2', '39C072_0', '39C082_0', '39C091_0', '39C131_0', '39C152_0', '39C162_0', '39C272_0', '39C282_0', '39C302_0', '39C312_0', '39C342_0', '39C372_0', '39C392_0', '39C402_0', '39C412_0', '39C442_0', '39C502_0', '39C522_0', '39C622_0', '39C712_0', '39C732_0', '39C742_0', '39P071_1', '39P132_2', '39P212_2', '39P412_2', '39P542_2', '39P642_2', '3C022_0', '3C041_0', '3C072_0', '3C082_0', '3C091_0', '3C101_0', '3C131_0', '3C152_0', '3C162_0', '3C172_0', '3C182_0', '3C202_0', '3C212_0', '3C222_0', '3C272_0', '3C282_0', '3C292_0', '3C302_0', '3C312_0', '3C322_0', '3C342_0', '3C372_0', '3C382_0', '3C392_0', '3C402_0', '3C412_0', '3C422_0', '3C432_0', '3C442_0', '3C472_0', '3C502_0', '3C522_0', '3C562_0', '3C582_0', '3C602_0', '3C612_0', '3C622_0', '3C632_0', '3C652_0', '3C672_0', '3C682_0', '3C712_0', '3C732_0', '3C742_0', '3P042_2', '3P052_2', '3P071_1', '3P092_2', '3P111_1', '3P132_2', '3P141_1', '3P162_2', '3P201_1', '3P212_2', '3P332_2', '3P352_2', '3P412_2', '3P422_2', '3P432_2', '3P452_2', '3P542_2', '3P642_2', '3P672_2', '3P712_2', '40C082_0', '40C091_0', '40C131_0', '40C152_0', '40C162_0', '40C272_0', '40C302_0', '40C312_0', '40C342_0', '40C372_0', '40C392_0', '40C402_0', '40C412_0', '40C442_0', '40C502_0', '40C522_0', '40C622_0', '40C712_0', '40C742_0', '40P071_1', '40P132_2', '40P212_2', '40P412_2', '40P542_2', '40P642_2', '41C082_0', '41C091_0', '41C131_0', '41C152_0', '41C162_0', '41C272_0', '41C302_0', '41C312_0', '41C342_0', '41C372_0', '41C392_0', '41C402_0', '41C412_0', '41C442_0', '41C502_0', '41C522_0', '41C622_0', '41C712_0', '41C742_0', '41P071_1', '41P132_2', '41P212_2', '41P412_2', '41P542_2', '42C082_0', '42C091_0', '42C131_0', '42C152_0', '42C162_0', '42C272_0', '42C302_0', '42C312_0', '42C342_0', '42C372_0', '42C392_0', '42C402_0', '42C412_0', '42C442_0', '42C502_0', '42C522_0', '42C622_0', '42C712_0', '42C742_0', '42P071_1', '42P132_2', '42P212_2', '42P412_2', '42P542_2', '43C082_0', '43C091_0', '43C131_0', '43C152_0', '43C162_0', '43C272_0', '43C302_0', '43C312_0', '43C342_0', '43C372_0', '43C392_0', '43C402_0', '43C412_0', '43C442_0', '43C502_0', '43C522_0', '43C622_0', '43C742_0', '43P071_1', '43P132_2', '43P212_2', '43P412_2', '43P542_2', '44C082_0', '44C091_0', '44C131_0', '44C152_0', '44C162_0', '44C272_0', '44C302_0', '44C312_0', '44C342_0', '44C372_0', '44C392_0', '44C402_0', '44C412_0', '44C442_0', '44C502_0', '44C522_0', '44C622_0', '44C742_0', '44P071_1', '44P132_2', '44P212_2', '44P412_2', '44P542_2', '45C082_0', '45C091_0', '45C131_0', '45C152_0', '45C162_0', '45C272_0', '45C302_0', '45C312_0', '45C342_0', '45C372_0', '45C392_0', '45C402_0', '45C412_0', '45C442_0', '45C502_0', '45C522_0', '45C742_0', '45P071_1', '45P132_2', '45P212_2', '45P412_2', '45P542_2', '46C082_0', '46C091_0', '46C152_0', '46C162_0', '46C272_0', '46C302_0', '46C312_0', '46C372_0', '46C392_0', '46C402_0', '46C412_0', '46C442_0', '46C502_0', '46C522_0', '46C742_0', '46P071_1', '46P132_2', '46P212_2', '46P412_2', '46P542_2', '47C082_0', '47C091_0', '47C152_0', '47C162_0', '47C272_0', '47C302_0', '47C312_0', '47C372_0', '47C392_0', '47C402_0', '47C412_0', '47C442_0', '47C502_0', '47C522_0', '47C742_0', '47P071_1', '47P132_2', '47P412_2', '47P542_2', '48C082_0', '48C091_0', '48C152_0', '48C162_0', '48C272_0', '48C302_0', '48C312_0', '48C372_0', '48C392_0', '48C402_0', '48C412_0', '48C442_0', '48C502_0', '48C522_0', '48C742_0', '48P071_1', '48P132_2', '48P412_2', '48P542_2', '49C082_0', '49C091_0', '49C152_0', '49C162_0', '49C272_0', '49C302_0', '49C312_0', '49C372_0', '49C392_0', '49C402_0', '49C412_0', '49C442_0', '49C502_0', '49C522_0', '49C742_0', '49P071_1', '49P132_2', '49P412_2', '49P542_2', '4C022_0', '4C041_0', '4C072_0', '4C082_0', '4C091_0', '4C101_0', '4C131_0', '4C152_0', '4C162_0', '4C172_0', '4C182_0', '4C202_0', '4C212_0', '4C222_0', '4C272_0', '4C282_0', '4C292_0', '4C302_0', '4C312_0', '4C322_0', '4C342_0', '4C372_0', '4C382_0', '4C392_0', '4C402_0', '4C412_0', '4C422_0', '4C432_0', '4C442_0', '4C472_0', '4C502_0', '4C522_0', '4C562_0', '4C582_0', '4C602_0', '4C612_0', '4C622_0', '4C632_0', '4C652_0', '4C672_0', '4C682_0', '4C712_0', '4C732_0', '4C742_0', '4P042_2', '4P052_2', '4P071_1', '4P092_2', '4P111_1', '4P132_2', '4P141_1', '4P162_2', '4P201_1', '4P212_2', '4P332_2', '4P352_2', '4P412_2', '4P422_2', '4P432_2', '4P452_2', '4P542_2', '4P642_2', '4P672_2', '4P712_2', '50C091_0', '50C152_0', '50C162_0', '50C272_0', '50C302_0', '50C312_0', '50C372_0', '50C392_0', '50C402_0', '50C412_0', '50C442_0', '50C502_0', '50C522_0', '50C742_0', '50P071_1', '50P132_2', '50P412_2', '50P542_2', '51C091_0', '51C152_0', '51C162_0', '51C272_0', '51C302_0', '51C312_0', '51C372_0', '51C392_0', '51C402_0', '51C412_0', '51C442_0', '51C502_0', '51C522_0', '51C742_0', '51P071_1', '51P132_2', '51P412_2', '51P542_2', '52C091_0', '52C152_0', '52C162_0', '52C272_0', '52C302_0', '52C312_0', '52C372_0', '52C392_0', '52C402_0', '52C442_0', '52C502_0', '52C522_0', '52C742_0', '52P071_1', '52P132_2', '52P412_2', '52P542_2', '53C091_0', '53C152_0', '53C162_0', '53C272_0', '53C302_0', '53C312_0', '53C372_0', '53C392_0', '53C402_0', '53C442_0', '53C502_0', '53C522_0', '53C742_0', '53P071_1', '53P132_2', '53P412_2', '53P542_2', '54C091_0', '54C152_0', '54C162_0', '54C272_0', '54C302_0', '54C312_0', '54C372_0', '54C392_0', '54C402_0', '54C442_0', '54C502_0', '54C522_0', '54C742_0', '54P071_1', '54P132_2', '54P412_2', '54P542_2', '55C091_0', '55C152_0', '55C162_0', '55C272_0', '55C302_0', '55C312_0', '55C372_0', '55C392_0', '55C402_0', '55C442_0', '55C502_0', '55C522_0', '55C742_0', '55P071_1', '55P132_2', '55P412_2', '55P542_2', '56C091_0', '56C152_0', '56C162_0', '56C272_0', '56C302_0', '56C312_0', '56C372_0', '56C392_0', '56C402_0', '56C442_0', '56C502_0', '56C522_0', '56C742_0', '56P071_1', '56P132_2', '56P412_2', '56P542_2', '57C091_0', '57C152_0', '57C162_0', '57C272_0', '57C302_0', '57C312_0', '57C372_0', '57C392_0', '57C402_0', '57C442_0', '57C502_0', '57C522_0', '57C742_0', '57P071_1', '57P132_2', '57P412_2', '57P542_2', '58C091_0', '58C152_0', '58C162_0', '58C272_0', '58C302_0', '58C312_0', '58C372_0', '58C392_0', '58C402_0', '58C442_0', '58C502_0', '58C522_0', '58C742_0', '58P071_1', '58P132_2', '58P412_2', '58P542_2', '59C091_0', '59C152_0', '59C162_0', '59C272_0', '59C302_0', '59C312_0', '59C372_0', '59C392_0', '59C402_0', '59C442_0', '59C502_0', '59C522_0', '59C742_0', '59P071_1', '59P132_2', '59P412_2', '59P542_2', '5C022_0', '5C041_0', '5C072_0', '5C082_0', '5C091_0', '5C101_0', '5C131_0', '5C152_0', '5C162_0', '5C172_0', '5C182_0', '5C202_0', '5C212_0', '5C222_0', '5C272_0', '5C282_0', '5C292_0', '5C302_0', '5C312_0', '5C322_0', '5C342_0', '5C372_0', '5C382_0', '5C392_0', '5C402_0', '5C412_0', '5C422_0', '5C432_0', '5C442_0', '5C502_0', '5C522_0', '5C562_0', '5C582_0', '5C602_0', '5C612_0', '5C622_0', '5C632_0', '5C652_0', '5C672_0', '5C682_0', '5C712_0', '5C732_0', '5C742_0', '5P042_2', '5P052_2', '5P071_1', '5P092_2', '5P111_1', '5P132_2', '5P141_1', '5P162_2', '5P201_1', '5P212_2', '5P332_2', '5P352_2', '5P412_2', '5P422_2', '5P432_2', '5P542_2', '5P642_2', '5P672_2', '5P712_2', '60C091_0', '60C152_0', '60C162_0', '60C272_0', '60C302_0', '60C312_0', '60C372_0', '60C392_0', '60C402_0', '60C442_0', '60C502_0', '60C522_0', '60C742_0', '60P071_1', '60P132_2', '60P412_2', '60P542_2', '61C091_0', '61C152_0', '61C162_0', '61C272_0', '61C302_0', '61C312_0', '61C372_0', '61C392_0', '61C402_0', '61C442_0', '61C502_0', '61C522_0', '61C742_0', '61P071_1', '61P132_2', '61P412_2', '61P542_2', '62C091_0', '62C152_0', '62C162_0', '62C272_0', '62C302_0', '62C312_0', '62C372_0', '62C392_0', '62C402_0', '62C442_0', '62C502_0', '62C522_0', '62C742_0', '62P071_1', '62P132_2', '62P412_2', '62P542_2', '63C091_0', '63C152_0', '63C162_0', '63C272_0', '63C302_0', '63C312_0', '63C372_0', '63C392_0', '63C402_0', '63C442_0', '63C502_0', '63C522_0', '63C742_0', '63P071_1', '63P132_2', '63P412_2', '63P542_2', '64C091_0', '64C152_0', '64C162_0', '64C272_0', '64C302_0', '64C312_0', '64C372_0', '64C392_0', '64C402_0', '64C442_0', '64C502_0', '64C522_0', '64C742_0', '64P071_1', '64P132_2', '64P412_2', '64P542_2', '65C091_0', '65C152_0', '65C162_0', '65C272_0', '65C302_0', '65C312_0', '65C372_0', '65C392_0', '65C402_0', '65C442_0', '65C502_0', '65C522_0', '65C742_0', '65P071_1', '65P132_2', '65P412_2', '65P542_2', '66C091_0', '66C152_0', '66C162_0', '66C272_0', '66C302_0', '66C312_0', '66C372_0', '66C392_0', '66C402_0', '66C442_0', '66C502_0', '66C522_0', '66C742_0', '66P071_1', '66P132_2', '66P412_2', '66P542_2', '67C091_0', '67C152_0', '67C162_0', '67C272_0', '67C302_0', '67C312_0', '67C372_0', '67C392_0', '67C402_0', '67C442_0', '67C502_0', '67C522_0', '67C742_0', '67P071_1', '67P132_2', '67P412_2', '67P542_2', '68C091_0', '68C152_0', '68C162_0', '68C272_0', '68C302_0', '68C312_0', '68C372_0', '68C392_0', '68C402_0', '68C442_0', '68C502_0', '68C522_0', '68C742_0', '68P071_1', '68P132_2', '68P412_2', '68P542_2', '69C091_0', '69C152_0', '69C162_0', '69C272_0', '69C302_0', '69C312_0', '69C372_0', '69C392_0', '69C402_0', '69C442_0', '69C502_0', '69C522_0', '69C742_0', '69P071_1', '69P132_2', '69P412_2', '6C022_0', '6C041_0', '6C072_0', '6C082_0', '6C091_0', '6C101_0', '6C131_0', '6C152_0', '6C162_0', '6C172_0', '6C182_0', '6C202_0', '6C212_0', '6C222_0', '6C272_0', '6C282_0', '6C292_0', '6C302_0', '6C312_0', '6C322_0', '6C342_0', '6C372_0', '6C382_0', '6C392_0', '6C402_0', '6C412_0', '6C422_0', '6C432_0', '6C442_0', '6C502_0', '6C522_0', '6C562_0', '6C582_0', '6C602_0', '6C612_0', '6C622_0', '6C652_0', '6C672_0', '6C682_0', '6C712_0', '6C732_0', '6C742_0', '6P042_2', '6P052_2', '6P071_1', '6P092_2', '6P111_1', '6P132_2', '6P141_1', '6P162_2', '6P201_1', '6P212_2', '6P332_2', '6P352_2', '6P412_2', '6P422_2', '6P432_2', '6P542_2', '6P642_2', '6P672_2', '6P712_2', '70C091_0', '70C152_0', '70C162_0', '70C272_0', '70C302_0', '70C312_0', '70C372_0', '70C392_0', '70C402_0', '70C442_0', '70C502_0', '70C522_0', '70C742_0', '70P071_1', '70P132_2', '70P412_2', '71C091_0', '71C152_0', '71C162_0', '71C272_0', '71C302_0', '71C312_0', '71C372_0', '71C392_0', '71C402_0', '71C442_0', '71C502_0', '71C522_0', '71C742_0', '71P071_1', '71P132_2', '71P412_2', '72C091_0', '72C152_0', '72C162_0', '72C272_0', '72C302_0', '72C312_0', '72C372_0', '72C392_0', '72C402_0', '72C442_0', '72C502_0', '72C522_0', '72C742_0', '72P071_1', '72P132_2', '72P412_2', '73C091_0', '73C152_0', '73C162_0', '73C272_0', '73C302_0', '73C312_0', '73C372_0', '73C392_0', '73C402_0', '73C442_0', '73C502_0', '73C522_0', '73C742_0', '73P071_1', '73P132_2', '73P412_2', '74C091_0', '74C152_0', '74C162_0', '74C272_0', '74C302_0', '74C312_0', '74C372_0', '74C392_0', '74C402_0', '74C442_0', '74C502_0', '74C522_0', '74C742_0', '74P071_1', '74P132_2', '74P412_2', '75C091_0', '75C152_0', '75C162_0', '75C272_0', '75C302_0', '75C312_0', '75C392_0', '75C402_0', '75C442_0', '75C502_0', '75C522_0', '75C742_0', '75P071_1', '75P132_2', '75P412_2', '76C091_0', '76C152_0', '76C162_0', '76C272_0', '76C302_0', '76C312_0', '76C392_0', '76C402_0', '76C442_0', '76C502_0', '76C522_0', '76C742_0', '76P071_1', '76P132_2', '76P412_2', '77C091_0', '77C152_0', '77C162_0', '77C272_0', '77C302_0', '77C312_0', '77C392_0', '77C402_0', '77C442_0', '77C502_0', '77C522_0', '77C742_0', '77P071_1', '77P412_2', '78C091_0', '78C152_0', '78C162_0', '78C272_0', '78C302_0', '78C312_0', '78C392_0', '78C402_0', '78C442_0', '78C502_0', '78C522_0', '78C742_0', '78P071_1', '78P412_2', '79C091_0', '79C152_0', '79C162_0', '79C272_0', '79C302_0', '79C312_0', '79C392_0', '79C402_0', '79C442_0', '79C502_0', '79C522_0', '79C742_0', '79P071_1', '79P412_2', '7C022_0', '7C041_0', '7C072_0', '7C082_0', '7C091_0', '7C101_0', '7C131_0', '7C152_0', '7C162_0', '7C172_0', '7C182_0', '7C202_0', '7C212_0', '7C222_0', '7C272_0', '7C282_0', '7C292_0', '7C302_0', '7C312_0', '7C322_0', '7C342_0', '7C372_0', '7C382_0', '7C392_0', '7C402_0', '7C412_0', '7C422_0', '7C432_0', '7C442_0', '7C502_0', '7C522_0', '7C562_0', '7C582_0', '7C602_0', '7C612_0', '7C622_0', '7C652_0', '7C672_0', '7C682_0', '7C712_0', '7C732_0', '7C742_0', '7P042_2', '7P052_2', '7P071_1', '7P092_2', '7P111_1', '7P132_2', '7P141_1', '7P162_2', '7P201_1', '7P212_2', '7P332_2', '7P352_2', '7P412_2', '7P422_2', '7P432_2', '7P542_2', '7P642_2', '7P672_2', '7P712_2', '80C091_0', '80C152_0', '80C162_0', '80C272_0', '80C302_0', '80C312_0', '80C392_0', '80C402_0', '80C442_0', '80C502_0', '80C522_0', '80C742_0', '80P071_1', '80P412_2', '81C091_0', '81C152_0', '81C162_0', '81C272_0', '81C302_0', '81C312_0', '81C392_0', '81C402_0', '81C442_0', '81C502_0', '81C522_0', '81C742_0', '81P071_1', '81P412_2', '82C091_0', '82C152_0', '82C162_0', '82C272_0', '82C302_0', '82C312_0', '82C392_0', '82C402_0', '82C442_0', '82C502_0', '82C522_0', '82C742_0', '82P071_1', '82P412_2', '83C091_0', '83C152_0', '83C162_0', '83C272_0', '83C302_0', '83C312_0', '83C392_0', '83C402_0', '83C442_0', '83C502_0', '83C522_0', '83C742_0', '83P071_1', '83P412_2', '84C091_0', '84C152_0', '84C162_0', '84C272_0', '84C302_0', '84C312_0', '84C392_0', '84C402_0', '84C442_0', '84C502_0', '84C522_0', '84C742_0', '84P071_1', '84P412_2', '85C091_0', '85C152_0', '85C162_0', '85C272_0', '85C302_0', '85C312_0', '85C392_0', '85C402_0', '85C442_0', '85C502_0', '85C522_0', '85C742_0', '85P071_1', '86C091_0', '86C152_0', '86C162_0', '86C272_0', '86C302_0', '86C312_0', '86C392_0', '86C402_0', '86C442_0', '86C502_0', '86C742_0', '86P071_1', '87C091_0', '87C152_0', '87C162_0', '87C272_0', '87C302_0', '87C312_0', '87C392_0', '87C402_0', '87C442_0', '87C502_0', '87C742_0', '87P071_1', '88C091_0', '88C152_0', '88C162_0', '88C272_0', '88C302_0', '88C312_0', '88C392_0', '88C402_0', '88C442_0', '88C502_0', '88C742_0', '88P071_1', '89C091_0', '89C152_0', '89C162_0', '89C272_0', '89C302_0', '89C312_0', '89C392_0', '89C402_0', '89C442_0', '89C502_0', '89C742_0', '89P071_1', '8C041_0', '8C072_0', '8C082_0', '8C091_0', '8C101_0', '8C131_0', '8C152_0', '8C162_0', '8C172_0', '8C182_0', '8C202_0', '8C212_0', '8C222_0', '8C272_0', '8C282_0', '8C292_0', '8C302_0', '8C312_0', '8C322_0', '8C342_0', '8C372_0', '8C382_0', '8C392_0', '8C402_0', '8C412_0', '8C422_0', '8C432_0', '8C442_0', '8C502_0', '8C522_0', '8C562_0', '8C582_0', '8C602_0', '8C612_0', '8C622_0', '8C652_0', '8C672_0', '8C682_0', '8C712_0', '8C732_0', '8C742_0', '8P042_2', '8P052_2', '8P071_1', '8P092_2', '8P111_1', '8P132_2', '8P141_1', '8P162_2', '8P201_1', '8P212_2', '8P332_2', '8P412_2', '8P422_2', '8P432_2', '8P542_2', '8P642_2', '8P672_2', '8P712_2', '90C091_0', '90C152_0', '90C162_0', '90C272_0', '90C302_0', '90C312_0', '90C392_0', '90C402_0', '90C442_0', '90C502_0', '90C742_0', '90P071_1', '91C091_0', '91C152_0', '91C162_0', '91C272_0', '91C302_0', '91C312_0', '91C392_0', '91C402_0', '91C442_0', '91C502_0', '91C742_0', '92C091_0', '92C152_0', '92C162_0', '92C272_0', '92C302_0', '92C312_0', '92C392_0', '92C402_0', '92C442_0', '92C502_0', '92C742_0', '93C091_0', '93C152_0', '93C162_0', '93C272_0', '93C302_0', '93C312_0', '93C392_0', '93C402_0', '93C502_0', '94C091_0', '94C152_0', '94C162_0', '94C272_0', '94C302_0', '94C312_0', '94C392_0', '94C402_0', '94C502_0', '95C091_0', '95C152_0', '95C162_0', '95C272_0', '95C302_0', '95C312_0', '95C392_0', '95C402_0', '95C502_0', '96C091_0', '96C152_0', '96C162_0', '96C272_0', '96C302_0', '96C312_0', '96C392_0', '96C402_0', '96C502_0', '97C091_0', '97C152_0', '97C162_0', '97C272_0', '97C302_0', '97C312_0', '97C392_0', '97C402_0', '97C502_0', '98C091_0', '98C152_0', '98C162_0', '98C272_0', '98C302_0', '98C312_0', '98C392_0', '98C402_0', '98C502_0', '99C091_0', '99C152_0', '99C162_0', '99C272_0', '99C302_0', '99C312_0', '99C392_0', '99C402_0', '99C502_0', '9C041_0', '9C072_0', '9C082_0', '9C091_0', '9C101_0', '9C131_0', '9C152_0', '9C162_0', '9C172_0', '9C182_0', '9C202_0', '9C212_0', '9C222_0', '9C272_0', '9C282_0', '9C292_0', '9C302_0', '9C312_0', '9C322_0', '9C342_0', '9C372_0', '9C382_0', '9C392_0', '9C402_0', '9C412_0', '9C422_0', '9C432_0', '9C442_0', '9C502_0', '9C522_0', '9C562_0', '9C582_0', '9C602_0', '9C612_0', '9C622_0', '9C652_0', '9C672_0', '9C682_0', '9C712_0', '9C732_0', '9C742_0', '9P052_2', '9P071_1', '9P111_1', '9P132_2', '9P141_1', '9P162_2', '9P201_1', '9P212_2', '9P332_2', '9P412_2', '9P422_2', '9P542_2', '9P642_2', '9P672_2', '9P712_2']
X = df_parameters.copy()
X = X.drop(["Label", "Label CONvsACUREM", "Label CONREMvsACU"], axis=1)
y = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
y_con_vs_acurem = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_conrem_vs_acu = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

def naivebayes(X,y, X_train, y_train, X_test):
    model = MultinomialNB()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores = cross_val_score(model, X, y, cv=5)
    print("---------- Naive Bayes ----------")
    return model, y_pred, scores

def supportvectormachine(X,y, X_train, y_train, X_test):
    model = SVC(kernel='linear', C=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores = cross_val_score(model, X, y, cv=5)
    print("---------- Support Vector Machine ----------","\n")
    return model, y_pred, scores

def classifierresults(X, y, y_test, y_pred, scores, model):
    print("Confusion Matrix:","\n",confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    seed = 7
    test_size = 0.33
    #kfold = KFold(n_splits=25, random_state=seed)
    #results = cross_val_score(model, X, y, cv=kfold)
    #print("K-Fold Cross Validation: Accuracy: %.3f%% (+/- %.3f%%)" % (results.mean()*100.0, results.std()*100.0))
    #loocv = LeaveOneOut()
    #results = cross_val_score(model, X, y, cv=loocv)
    #print("Leave-One-Out Cross Validation: Accuracy: %.3f%% (+/- %.3f%%)" % (results.mean()*100.0, results.std()*100.0))
    kfold = ShuffleSplit(n_splits=10, test_size=test_size, random_state=seed)
    results = cross_val_score(model, X, y, cv=kfold)
    print("Repeated Random Test-Train Splits: Accuracy: %.3f%% (+/- %.3f%%)" % (results.mean()*100.0, results.std()*100.0))

## Word Graph Analysis without word windows (complete, raw input)

In [2]:
task = "Taaloefening 1"
task2 = "Taaloefening 2"
filenames = []
for participant,row in df.iterrows():
    if isinstance(df[task][participant], float) is not True:
        filename = "1_" + participant + str(row[0])
        #text_file = open(filename, "w")
        #text_file.write(df[task][participant])
        #text_file.close()
        filenames.append(filename)
    if isinstance(df[task2][participant], float) is not True:
        filename = "2_" + participant + str(row[0])
        #text_file = open(filename, "w")
        #text_file.write(df[task2][participant])
        #text_file.close()
        filenames.append(filename)
filenames = list(set(filenames))
filenames.sort()
filenames1 = filenames[:84]
filenames2 = filenames[84:]

y=[]
y_con_vs_acurem = []
y_conrem_vs_acu = []
y1=[]
y_con_vs_acurem1 = []
y_conrem_vs_acu1 = []
y2=[]
y_con_vs_acurem2 = []
y_conrem_vs_acu2 = []
        
for file in filenames:
    # controls
    if file.endswith('0'):
        y.append(0)
        y_con_vs_acurem.append(0)
        y_conrem_vs_acu.append(0)
        # task 1
        if file.startswith('1'):
            y1.append(0)
            y_con_vs_acurem1.append(0)
            y_conrem_vs_acu1.append(0)
        # task 2
        elif file.startswith('2'):
            y2.append(0)
            y_con_vs_acurem2.append(0)
            y_conrem_vs_acu2.append(0)
    # acute patients        
    elif file.endswith('1'):
        y.append(1)
        y_con_vs_acurem.append(1)
        y_conrem_vs_acu.append(1)
        
        # task 1
        if file.startswith('1'):
            y1.append(1)
            y_con_vs_acurem1.append(1)
            y_conrem_vs_acu1.append(1)
        # task 2
        elif file.startswith('2'):
            y2.append(1)
            y_con_vs_acurem2.append(1)
            y_conrem_vs_acu2.append(1)
            
    # remission patients
    elif file.endswith('2'):
        y.append(2)
        y_con_vs_acurem.append(1)
        y_conrem_vs_acu.append(0)
        
        # task 1
        if file.startswith('1'):
            y1.append(2)
            y_con_vs_acurem1.append(1)
            y_conrem_vs_acu1.append(0)
        # task 2
        elif file.startswith('2'):
            y2.append(2)
            y_con_vs_acurem2.append(1)
            y_conrem_vs_acu2.append(0)

### The following lines read in the tables "params_table" as created by the SpeechGraph Software, extracts the features and stores them in a table

### Naive Bayes and Support Vector Machines raw input (no windows used) (classification and report)

In [3]:
X1 = df_parameters_task1.copy()
X1 = X1.drop(["Label", "Label CONvsACUREM","Label CONREMvsACU" ], axis =1)
X2 = df_parameters_task2.copy()
X2 = X2.drop(["Label", "Label CONvsACUREM","Label CONREMvsACU" ], axis = 1)
X_both = X1.append(X2)
y_both = y1+y2
y_con_vs_acurem_both = y_con_vs_acurem1 + y_con_vs_acurem2
y_conrem_vs_acu_both = y_conrem_vs_acu1 + y_conrem_vs_acu2

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_both, y_both, test_size = 0.20)

In [360]:
model, y_pred, scores = naivebayes(X_both,y_both,X_train, y_train, X_test)
classifierresults(X_both, y_both, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X_both,y_both,X_train, y_train, X_test)
classifierresults(X_both, y_both, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[10  4  5]
 [ 2  2  4]
 [ 2  3  1]]
              precision    recall  f1-score   support

           0       0.71      0.53      0.61        19
           1       0.22      0.25      0.24         8
           2       0.10      0.17      0.12         6

    accuracy                           0.39        33
   macro avg       0.35      0.31      0.32        33
weighted avg       0.48      0.39      0.43        33

Accuracy: 0.51 (+/- 0.19)
Repeated Random Test-Train Splits: Accuracy: 47.778% (+/- 4.596%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[18  0  1]
 [ 7  0  1]
 [ 6  0  0]]
              precision    recall  f1-score   support

           0       0.58      0.95      0.72        19
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         6

    accuracy                           0.55        33
   macro avg       0.19      0.32      0.24        33
weight

/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 59.444% (+/- 5.459%)


In [275]:
X_train, X_test, y_train, y_test = train_test_split(X_both, y_con_vs_acurem_both, test_size = 0.20)
model, y_pred, scores = naivebayes(X_both,y_con_vs_acurem_both,X_train, y_train, X_test)
classifierresults(X_both, y_con_vs_acurem_both, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X_both,y_con_vs_acurem_both,X_train, y_train, X_test)
classifierresults(X_both, y_con_vs_acurem_both, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[ 4 16]
 [ 2 11]]
              precision    recall  f1-score   support

           0       0.67      0.20      0.31        20
           1       0.41      0.85      0.55        13

    accuracy                           0.45        33
   macro avg       0.54      0.52      0.43        33
weighted avg       0.56      0.45      0.40        33

Accuracy: 0.63 (+/- 0.17)
Repeated Random Test-Train Splits: Accuracy: 60.926% (+/- 5.135%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[13  7]
 [ 7  6]]
              precision    recall  f1-score   support

           0       0.65      0.65      0.65        20
           1       0.46      0.46      0.46        13

    accuracy                           0.58        33
   macro avg       0.56      0.56      0.56        33
weighted avg       0.58      0.58      0.58        33

Accuracy: 0.68 (+/- 0.09)
Repeated Random Test-Train Splits: Accuracy: 61.481% (+/- 6.186%)


In [276]:
X_train, X_test, y_train, y_test = train_test_split(X_both, y_conrem_vs_acu_both, test_size = 0.20)
model, y_pred, scores = naivebayes(X_both,y_conrem_vs_acu_both,X_train, y_train,X_test)
classifierresults(X_both, y_conrem_vs_acu_both, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X_both,y_conrem_vs_acu_both,X_train, y_train, X_test)
classifierresults(X_both, y_conrem_vs_acu_both, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[22  5]
 [ 2  4]]
              precision    recall  f1-score   support

           0       0.92      0.81      0.86        27
           1       0.44      0.67      0.53         6

    accuracy                           0.79        33
   macro avg       0.68      0.74      0.70        33
weighted avg       0.83      0.79      0.80        33

Accuracy: 0.61 (+/- 0.25)
Repeated Random Test-Train Splits: Accuracy: 56.111% (+/- 4.540%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[27  0]
 [ 6  0]]
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        27
           1       0.00      0.00      0.00         6

    accuracy                           0.82        33
   macro avg       0.41      0.50      0.45        33
weighted avg       0.67      0.82      0.74        33

Accuracy: 0.79 (+/- 0.01)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 78.333% (+/- 3.976%)


In [277]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.20)
model, y_pred, scores = naivebayes(X1,y1,X_train, y_train, X_test)
classifierresults(X1, y1, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X1,y1,X_train, y_train, X_test)
classifierresults(X1, y1, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[4 4 1]
 [2 3 2]
 [0 0 1]]
              precision    recall  f1-score   support

           0       0.67      0.44      0.53         9
           1       0.43      0.43      0.43         7
           2       0.25      1.00      0.40         1

    accuracy                           0.47        17
   macro avg       0.45      0.62      0.45        17
weighted avg       0.54      0.47      0.48        17

Accuracy: 0.44 (+/- 0.12)
Repeated Random Test-Train Splits: Accuracy: 52.500% (+/- 9.320%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[8 1 0]
 [7 0 0]
 [1 0 0]]
              precision    recall  f1-score   support

           0       0.50      0.89      0.64         9
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         1

    accuracy                           0.47        17
   macro avg       0.17      0.30      0.21        17
weighted avg       0.26      0.47      0.34        17

Accuracy: 0.59 (+/- 0.09)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 52.857% (+/- 3.113%)


In [278]:
X_train, X_test, y_train, y_test = train_test_split(X1, y_con_vs_acurem1, test_size = 0.20)
model, y_pred, scores = naivebayes(X1,y_con_vs_acurem1,X_train, y_train, X_test)
classifierresults(X1, y_con_vs_acurem1, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X1,y_con_vs_acurem1,X_train, y_train, X_test)
classifierresults(X1, y_con_vs_acurem1, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[7 3]
 [1 6]]
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.67      0.86      0.75         7

    accuracy                           0.76        17
   macro avg       0.77      0.78      0.76        17
weighted avg       0.79      0.76      0.77        17

Accuracy: 0.57 (+/- 0.19)
Repeated Random Test-Train Splits: Accuracy: 64.286% (+/- 9.035%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[9 1]
 [4 3]]
              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.75      0.43      0.55         7

    accuracy                           0.71        17
   macro avg       0.72      0.66      0.66        17
weighted avg       0.72      0.71      0.68        17

Accuracy: 0.59 (+/- 0.16)
Repeated Random Test-Train Splits: Accuracy: 58.571% (+/- 7.178%)


In [279]:
X_train, X_test, y_train, y_test = train_test_split(X1, y_conrem_vs_acu1, test_size = 0.20)
model, y_pred, scores = naivebayes(X1,y_conrem_vs_acu1,X_train, y_train, X_test)
classifierresults(X1, y_conrem_vs_acu1, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X1,y_conrem_vs_acu1,X_train, y_train, X_test)
classifierresults(X1, y_conrem_vs_acu1, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[5 7]
 [0 5]]
              precision    recall  f1-score   support

           0       1.00      0.42      0.59        12
           1       0.42      1.00      0.59         5

    accuracy                           0.59        17
   macro avg       0.71      0.71      0.59        17
weighted avg       0.83      0.59      0.59        17

Accuracy: 0.52 (+/- 0.21)
Repeated Random Test-Train Splits: Accuracy: 59.643% (+/- 6.971%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[12  0]
 [ 5  0]]
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        12
           1       0.00      0.00      0.00         5

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17

Accuracy: 0.65 (+/- 0.05)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 61.786% (+/- 3.589%)


In [280]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size = 0.20)
model, y_pred, scores = naivebayes(X2,y2,X_train, y_train, X_test)
classifierresults(X2, y2, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X2,y2,X_train, y_train, X_test)
classifierresults(X2, y2, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[5 0 5]
 [0 0 1]
 [0 0 5]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        10
           1       0.00      0.00      0.00         1
           2       0.45      1.00      0.62         5

    accuracy                           0.62        16
   macro avg       0.48      0.50      0.43        16
weighted avg       0.77      0.62      0.61        16

Accuracy: 0.62 (+/- 0.11)
Repeated Random Test-Train Splits: Accuracy: 60.370% (+/- 10.351%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[8 0 2]
 [1 0 0]
 [3 0 2]]
              precision    recall  f1-score   support

           0       0.67      0.80      0.73        10
           1       0.00      0.00      0.00         1
           2       0.50      0.40      0.44         5

    accuracy                           0.62        16
   macro avg       0.39      0.40      0.39        16
weighted avg       0.57      0.62      0.59        16

Accuracy: 0.67 (+/- 0.10)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 61.111% (+/- 8.646%)


In [281]:
X_train, X_test, y_train, y_test = train_test_split(X2, y_con_vs_acurem2, test_size = 0.20)
model, y_pred, scores = naivebayes(X2,y_con_vs_acurem2,X_train, y_train, X_test)
classifierresults(X2, y_con_vs_acurem2, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X2,y_con_vs_acurem2,X_train, y_train, X_test)
classifierresults(X2, y_con_vs_acurem2, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[8 2]
 [2 4]]
              precision    recall  f1-score   support

           0       0.80      0.80      0.80        10
           1       0.67      0.67      0.67         6

    accuracy                           0.75        16
   macro avg       0.73      0.73      0.73        16
weighted avg       0.75      0.75      0.75        16

Accuracy: 0.66 (+/- 0.14)
Repeated Random Test-Train Splits: Accuracy: 65.556% (+/- 6.839%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[9 1]
 [4 2]]
              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.67      0.33      0.44         6

    accuracy                           0.69        16
   macro avg       0.68      0.62      0.61        16
weighted avg       0.68      0.69      0.66        16

Accuracy: 0.66 (+/- 0.26)
Repeated Random Test-Train Splits: Accuracy: 72.593% (+/- 4.743%)


In [282]:
X_train, X_test, y_train, y_test = train_test_split(X2, y_conrem_vs_acu2, test_size = 0.20)
model, y_pred, scores = naivebayes(X2,y_conrem_vs_acu2,X_train, y_train, X_test)
classifierresults(X2, y_conrem_vs_acu2, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X2,y_conrem_vs_acu2,X_train, y_train, X_test)
classifierresults(X2, y_conrem_vs_acu2, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[15  0]
 [ 1  0]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       0.00      0.00      0.00         1

    accuracy                           0.94        16
   macro avg       0.47      0.50      0.48        16
weighted avg       0.88      0.94      0.91        16

Accuracy: 0.81 (+/- 0.18)
Repeated Random Test-Train Splits: Accuracy: 84.815% (+/- 4.521%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[15  0]
 [ 1  0]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       0.00      0.00      0.00         1

    accuracy                           0.94        16
   macro avg       0.47      0.50      0.48        16
weighted avg       0.88      0.94      0.91        16

Accuracy: 0.91 (+/- 0.11)
Repeated Random Test-Train Splits: Accuracy: 88.889% (+/- 8.765%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Word Graph Analysis with 30-word-windows
Creates the dataframes with 30-word-windows

In [368]:
df_movingwindows.head()

,Participant,ID,Label,Text,Lemmas,Task
0,P642,0,2,Terwijl de kok zijn deeg aan het voorbereiden ...,terwijl de kok zijn deeg aan_het voorbereiden ...,2
1,P642,1,2,de kok zijn deeg aan het voorbereiden is staat...,de kok zijn deeg aan_het voorbereiden zijn sta...,2
2,P642,2,2,kok zijn deeg aan het voorbereiden is staat er...,kok zijn deeg aan_het voorbereiden zijn staan ...,2
3,P642,3,2,zijn deeg aan het voorbereiden is staat er iem...,zijn deeg aan_het voorbereiden zijn staan er i...,2
4,P642,4,2,deeg aan het voorbereiden is staat er iemand t...,deeg aan_het voorbereiden zijn staan er iemand...,2


In [4]:
filenames1_mw=[]
for index, row in df_movingwindows.iterrows():
    if df_movingwindows.loc[index]["Task"] == 1:
        if isinstance(df_movingwindows.loc[index, "Text"], float) is not True:
            filename = str(row[1]) + str(row[0]) + "_" + str(row[2])
            #text_file = open(filename, "w")
            #text_file.write(df_movingwindows.loc[index]["Text"])
            #text_file.close()
            filenames1_mw.append(filename)
            
filenames2_mw=[]
for index, row in df_movingwindows.iterrows():
    if df_movingwindows.loc[index]["Task"] == 2:
        if isinstance(df_movingwindows.loc[index, "Text"], float) is not True:
            filename = str(row[1]) + str(row[0]) + "_" + str(row[2])
            #text_file = open(filename, "w")
            #text_file.write(df_movingwindows.loc[index]["Text"])
            #text_file.close()
            filenames2_mw.append(filename)
filenames1_mw.sort()
filenames2_mw.sort()

y1_mw = []
y_con_vs_acurem1_mw = []
y_conrem_vs_acu1_mw = []

for file in filenames1_mw:
    if file.endswith('0'):
        y1_mw.append(0)
        y_con_vs_acurem1_mw.append(0)
        y_conrem_vs_acu1_mw.append(0)
    elif file.endswith('1'):
        y1_mw.append(1)
        y_con_vs_acurem1_mw.append(1)
        y_conrem_vs_acu1_mw.append(1)
    elif file.endswith('2'):
        y1_mw.append(2)
        y_con_vs_acurem1_mw.append(1)
        y_conrem_vs_acu1_mw.append(0)
        
        
        
y2_mw = []
y_con_vs_acurem2_mw = []
y_conrem_vs_acu2_mw = []

for file in filenames2_mw:
    if file.endswith('0'):
        y2_mw.append(0)
        y_con_vs_acurem2_mw.append(0)
        y_conrem_vs_acu2_mw.append(0)
    elif file.endswith('1'):
        y2_mw.append(1)
        y_con_vs_acurem2_mw.append(1)
        y_conrem_vs_acu2_mw.append(1)
    elif file.endswith('2'):
        y2_mw.append(2)
        y_con_vs_acurem2_mw.append(1)
        y_conrem_vs_acu2_mw.append(0)
        
y_both_mw = y1_mw + y2_mw
y_con_vs_acurem_both_mw = y_con_vs_acurem1_mw + y_con_vs_acurem2_mw
y_conrem_vs_acu_both_mw = y_conrem_vs_acu1_mw + y_conrem_vs_acu2_mw
df_parameters_windows_both = df_parameters_windows_task1.append(df_parameters_windows_task2)

X1_mw = df_parameters_windows_task1.copy()
X1_mw = np.nan_to_num(X1_mw)
y1_mw = np.nan_to_num(y1_mw)
X2_mw = df_parameters_windows_task2.copy()
X2_mw = np.nan_to_num(X2_mw)
y2_mw = np.nan_to_num(y2_mw)
X_both_mw = df_parameters_windows_both.copy()
X_both_mw = np.nan_to_num(X_both_mw)
y_both_mw = np.nan_to_num(y_both_mw)

### Naive Bayes and Support Vector Machines on 30-word-windows (classification and report)

In [371]:
X_train, X_test, y_train, y_test = train_test_split(X1_mw, y1_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X1_mw,y1_mw,X_train, y_train, X_test)
classifierresults(X1_mw, y1_mw, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X1,y1,X_train, y_train, X_test)
classifierresults(X1_mw, y1_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[710   0  61]
 [265   0  13]
 [  5   0   5]]
              precision    recall  f1-score   support

           0       0.72      0.92      0.81       771
           1       0.00      0.00      0.00       278
           2       0.06      0.50      0.11        10

    accuracy                           0.68      1059
   macro avg       0.26      0.47      0.31      1059
weighted avg       0.53      0.68      0.59      1059

Accuracy: 0.68 (+/- 0.08)
Repeated Random Test-Train Splits: Accuracy: 68.169% (+/- 0.963%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[771   0   0]
 [278   0   0]
 [ 10   0   0]]
              precision    recall  f1-score   support

           0       0.73      1.00      0.84       771
           1       0.00      0.00      0.00       278
           2       0.00      0.00      0.00        10

    accuracy                           0.73      1059
   macro avg       0.24      0.33      0.28      1059
weighted avg       0.53      0.73      0.61      1059

Accuracy: 0.59 (+/- 0.09)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 73.793% (+/- 0.777%)


In [212]:
X_train, X_test, y_train, y_test = train_test_split(X1_mw, y_con_vs_acurem1_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X1_mw,y_con_vs_acurem1_mw,X_train, y_train, X_test)
classifierresults(X1_mw, y_con_vs_acurem1_mw, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X1_mw,y_con_vs_acurem1_mw,X_train, y_train, X_test)
classifierresults(X1_mw, y_con_vs_acurem1_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[448 354]
 [127 130]]
              precision    recall  f1-score   support

           0       0.78      0.56      0.65       802
           1       0.27      0.51      0.35       257

    accuracy                           0.55      1059
   macro avg       0.52      0.53      0.50      1059
weighted avg       0.66      0.55      0.58      1059

Accuracy: 0.56 (+/- 0.49)
Repeated Random Test-Train Splits: Accuracy: 55.915% (+/- 1.753%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[785  17]
 [248   9]]
              precision    recall  f1-score   support

           0       0.76      0.98      0.86       802
           1       0.35      0.04      0.06       257

    accuracy                           0.75      1059
   macro avg       0.55      0.51      0.46      1059
weighted avg       0.66      0.75      0.66      1059

Accuracy: 0.72 (+/- 0.05)
Repeated Random Test-Train Splits: Accuracy: 72.740% (+/- 1.0

In [213]:
X_train, X_test, y_train, y_test = train_test_split(X1_mw, y_conrem_vs_acu1_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X1_mw,y_conrem_vs_acu1_mw,X_train, y_train, X_test)
classifierresults(X1_mw, y_conrem_vs_acu1_mw, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X1_mw,y_conrem_vs_acu1_mw,X_train, y_train, X_test)
classifierresults(X1_mw, y_conrem_vs_acu1_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[440 352]
 [130 137]]
              precision    recall  f1-score   support

           0       0.77      0.56      0.65       792
           1       0.28      0.51      0.36       267

    accuracy                           0.54      1059
   macro avg       0.53      0.53      0.50      1059
weighted avg       0.65      0.54      0.57      1059

Accuracy: 0.56 (+/- 0.51)
Repeated Random Test-Train Splits: Accuracy: 56.104% (+/- 1.642%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[773  19]
 [259   8]]
              precision    recall  f1-score   support

           0       0.75      0.98      0.85       792
           1       0.30      0.03      0.05       267

    accuracy                           0.74      1059
   macro avg       0.52      0.50      0.45      1059
weighted avg       0.63      0.74      0.65      1059

Accuracy: 0.73 (+/- 0.05)
Repeated Random Test-Train Splits: Accuracy: 73.930% (+/- 0.8

In [214]:
X_train, X_test, y_train, y_test = train_test_split(X2_mw, y2_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X2_mw,y2_mw,X_train, y_train, X_test)
classifierresults(X2_mw, y2_mw, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X2_mw,y2_mw,X_train, y_train, X_test)
classifierresults(X2_mw, y2_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[311 191  16]
 [ 13  11   0]
 [ 55  38   7]]
              precision    recall  f1-score   support

           0       0.82      0.60      0.69       518
           1       0.05      0.46      0.08        24
           2       0.30      0.07      0.11       100

    accuracy                           0.51       642
   macro avg       0.39      0.38      0.30       642
weighted avg       0.71      0.51      0.58       642

Accuracy: 0.49 (+/- 0.72)
Repeated Random Test-Train Splits: Accuracy: 51.840% (+/- 2.442%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[516   0   2]
 [ 24   0   0]
 [100   0   0]]
              precision    recall  f1-score   support

           0       0.81      1.00      0.89       518
           1       0.00      0.00      0.00        24
           2       0.00      0.00      0.00       100

    accuracy                           0.80       642
   macro avg       0.27      0.33      0.3

/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 79.151% (+/- 0.964%)


In [215]:
X_train, X_test, y_train, y_test = train_test_split(X2_mw, y_con_vs_acurem2_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X2_mw,y_con_vs_acurem2_mw,X_train, y_train, X_test)
classifierresults(X2_mw, y_con_vs_acurem2_mw, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X2_mw,y_con_vs_acurem2_mw,X_train, y_train, X_test)
classifierresults(X2_mw, y_con_vs_acurem2_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[307 210]
 [ 74  51]]
              precision    recall  f1-score   support

           0       0.81      0.59      0.68       517
           1       0.20      0.41      0.26       125

    accuracy                           0.56       642
   macro avg       0.50      0.50      0.47       642
weighted avg       0.69      0.56      0.60       642

Accuracy: 0.55 (+/- 0.58)
Repeated Random Test-Train Splits: Accuracy: 57.632% (+/- 2.951%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[506  11]
 [121   4]]
              precision    recall  f1-score   support

           0       0.81      0.98      0.88       517
           1       0.27      0.03      0.06       125

    accuracy                           0.79       642
   macro avg       0.54      0.51      0.47       642
weighted avg       0.70      0.79      0.72       642

Accuracy: 0.78 (+/- 0.06)
Repeated Random Test-Train Splits: Accuracy: 79.217% (+/- 0.9

In [216]:
X_train, X_test, y_train, y_test = train_test_split(X2_mw, y_conrem_vs_acu2_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X2_mw,y_conrem_vs_acu2_mw,X_train, y_train, X_test)
classifierresults(X2_mw, y_conrem_vs_acu2_mw, y_test, y_pred, scores, model)
model, y_pred, scores = supportvectormachine(X2_mw,y_conrem_vs_acu2_mw,X_train, y_train, X_test)
classifierresults(X2_mw, y_conrem_vs_acu2_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[406 214]
 [ 14   8]]
              precision    recall  f1-score   support

           0       0.97      0.65      0.78       620
           1       0.04      0.36      0.07        22

    accuracy                           0.64       642
   macro avg       0.50      0.51      0.42       642
weighted avg       0.93      0.64      0.76       642

Accuracy: 0.59 (+/- 0.88)
Repeated Random Test-Train Splits: Accuracy: 64.453% (+/- 2.990%)
---------- Support Vector Machine ---------- 

Confusion Matrix: 
 [[620   0]
 [ 22   0]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       620
           1       0.00      0.00      0.00        22

    accuracy                           0.97       642
   macro avg       0.48      0.50      0.49       642
weighted avg       0.93      0.97      0.95       642

Accuracy: 0.95 (+/- 0.01)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Repeated Random Test-Train Splits: Accuracy: 95.642% (+/- 0.490%)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_both_mw, y_both_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X_both_mw,y_both_mw,X_train, y_train, X_test)
classifierresults(X_both_mw, y_both_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[1290    0    4]
 [ 303    0    4]
 [  97    0    3]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      1294
           1       0.00      0.00      0.00       307
           2       0.27      0.03      0.05       100

    accuracy                           0.76      1701
   macro avg       0.35      0.34      0.31      1701
weighted avg       0.60      0.76      0.66      1701

Accuracy: 0.76 (+/- 0.01)
Repeated Random Test-Train Splits: Accuracy: 76.192% (+/- 0.527%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# never compiled completely (both language tasks on support vector machine, C vs PA vs PR)
X_train, X_test, y_train, y_test = train_test_split(X_both_mw, y_both_mw, test_size = 0.20)
model, y_pred, scores = supportvectormachine(X_both_mw,y_both_mw,X_train, y_train, X_test)
classifierresults(X_both_mw, y_both_mw, y_test, y_pred, scores, model)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_both_mw, y_con_vs_acurem_both_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X_both_mw,y_con_vs_acurem_both_mw,X_train, y_train, X_test)
classifierresults(X_both_mw, y_con_vs_acurem_both_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[1288    8]
 [ 401    4]]
              precision    recall  f1-score   support

           0       0.76      0.99      0.86      1296
           1       0.33      0.01      0.02       405

    accuracy                           0.76      1701
   macro avg       0.55      0.50      0.44      1701
weighted avg       0.66      0.76      0.66      1701

Accuracy: 0.76 (+/- 0.01)
Repeated Random Test-Train Splits: Accuracy: 76.152% (+/- 0.476%)


In [ ]:
# never compiled completely (both language tasks on support vector machine, C vs PA+PR)
X_train, X_test, y_train, y_test = train_test_split(X_both_mw, y_con_vs_acurem_both_mw, test_size = 0.20)
model, y_pred, scores = supportvectormachine(X_both_mw,y_con_vs_acurem_both_mw,X_train, y_train, X_test)
classifierresults(X_both_mw, y_con_vs_acurem_both_mw, y_test, y_pred, scores, model)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_both_mw, y_conrem_vs_acu_both_mw, test_size = 0.20)
model, y_pred, scores = naivebayes(X_both_mw,y_conrem_vs_acu_both_mw,X_train, y_train, X_test)
classifierresults(X_both_mw, y_conrem_vs_acu_both_mw, y_test, y_pred, scores, model)

---------- Naive Bayes ----------
Confusion Matrix: 
 [[1409    0]
 [ 292    0]]
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      1409
           1       0.00      0.00      0.00       292

    accuracy                           0.83      1701
   macro avg       0.41      0.50      0.45      1701
weighted avg       0.69      0.83      0.75      1701

Accuracy: 0.83 (+/- 0.00)
Repeated Random Test-Train Splits: Accuracy: 82.558% (+/- 0.512%)


/Users/lisabecker/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# never compiled fully (both language tasks on support vector machine, C+PR vs PA)
X_train, X_test, y_train, y_test = train_test_split(X_both_mw, y_conrem_vs_acu_both_mw, test_size = 0.20)
model, y_pred, scores = supportvectormachine(X_both_mw,y_conrem_vs_acu_both_mw,X_train, y_train, X_test)
classifierresults(X_both_mw, y_conrem_vs_acu_both_mw, y_test, y_pred, scores, model)